### Loading Libraries

In [1]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import mysql.connector
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM
import lightgbm as lgb
import pickle
import matplotlib.pyplot as plt 
import seaborn 
from tqdm import tqdm

## SQL Injection Model

### Fetching SQL Data from Database

In [2]:
con = mysql.connector.connect(
    host = "localhost",
    user = "root",
    passwd = "G5thesis",
    database = "dataset_NF"
)
cursor = con.cursor()

In [3]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"

In [4]:
cursor.execute(q1)

In [5]:
colnames = [row[0] for row in cursor.fetchall()]

In [6]:
colnames

['IN_BYTES',
 'IN_PKTS',
 'OUT_BYTES',
 'OUT_PKTS',
 'TCP_FLAGS',
 'CLIENT_TCP_FLAGS',
 'SERVER_TCP_FLAGS',
 'FLOW_DURATION_MILLISECONDS',
 'DURATION_IN',
 'DURATION_OUT',
 'MIN_TTL',
 'MAX_TTL',
 'LONGEST_FLOW_PKT',
 'SHORTEST_FLOW_PKT',
 'MIN_IP_PKT_LEN',
 'MAX_IP_PKT_LEN',
 'SRC_TO_DST_SECOND_BYTES',
 'DST_TO_SRC_SECOND_BYTES',
 'SRC_TO_DST_AVG_THROUGHPUT',
 'DST_TO_SRC_AVG_THROUGHPUT',
 'NUM_PKTS_UP_TO_128_BYTES',
 'NUM_PKTS_128_TO_256_BYTES',
 'NUM_PKTS_256_TO_512_BYTES',
 'NUM_PKTS_512_TO_1024_BYTES',
 'NUM_PKTS_1024_TO_1514_BYTES',
 'TCP_WIN_MAX_IN',
 'TCP_WIN_MAX_OUT',
 'DNS_TTL_ANSWER',
 'Attack',
 'L4_DST_PORT_21',
 'L4_DST_PORT_22',
 'L4_DST_PORT_67',
 'L4_DST_PORT_80',
 'L4_DST_PORT_443',
 'L4_DST_PORT_500',
 'L4_DST_PORT_3306',
 'L4_DST_PORT_8080',
 'L4_DST_PORT_31808',
 'L4_DST_PORT_OTHERS',
 'L7_PROTO_7',
 'L7_PROTO_79',
 'L7_PROTO_92',
 'L7_PROTO_131',
 'L7_PROTO_178',
 'L7_PROTO_OTHERS']

## Trial Starting

In [6]:
# Extract all the data
labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() """,(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [7]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1785,21,25338,33,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,707,6,528,5,30,26,30,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,3375,14,6999,14,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1254,22,9753,25,31,31,30,4294951,16,16,...,0,0,0,0,0,0,0,0,0,1
4,707,6,528,5,30,26,30,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [9]:
for i in tqdm(range(0, 15)):
    df_copy.truncate(before=-1, after=-1)
    df_copy = df.copy()
    df_copy['Attack'][df_copy['Attack'] != i] = 0
    df_copy.replace({'Attack': i}, {'Attack': 1}, inplace=True)
    X_copy = df_copy.drop(columns=['Attack'])
    Y_copy = df_copy[['Attack']]
    X_train_copy, X_test_copy, y_train_copy, y_test_copy = train_test_split(X_copy, Y_copy, test_size=0.2, random_state=30)
    base_model = BalancedRandomForestClassifier(n_estimators=100, random_state=30, class_weight='balanced')
    base_model.fit(X_train_copy , y_train_copy )
    y_pred_copy = base_model.predict(X_test_copy)
    base_models.append(('model_' + str(i), base_model))
    results_copy = classification_report(y_test_copy,y_pred_copy,output_dict = True)
    df_res = pd.DataFrame(results_copy).transpose()
    print(i)
    print(df_res)
    print("\n")

  0%|                                                    | 0/15 [00:00<?, ?it/s]


NameError: name 'df_copy' is not defined

## Trial Ending

In [70]:
query_SQLInjection = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 13"

In [71]:
cursor.execute(query_SQLInjection)

In [72]:
data = cursor.fetchall()

In [73]:
df_SQL = pd.DataFrame(data = data, columns = colnames)

In [74]:
df_SQL.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,478,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,0,1,0,0,0,0,0
1,478,5,1147,5,219,219,27,4294935,31,31,...,0,0,0,0,1,0,0,0,0,0
2,471,5,1147,5,219,219,27,4294945,21,21,...,0,0,0,0,1,0,0,0,0,0
3,461,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,0,1,0,0,0,0,0
4,486,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,0,1,0,0,0,0,0


In [75]:
df_SQL.shape 

(354, 45)

### Fetching Other Types from Database

In [76]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 500000"

In [77]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [78]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,3045,17,4171,17,222,222,90,4294874,93,93,...,0,0,0,1,0,0,0,0,0,1
1,2338,28,90814,78,223,219,31,4294951,16,16,...,0,0,0,0,0,0,0,0,0,1
2,74,1,90,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,62,1,139,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,93,1,158,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [79]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [80]:
df = pd.concat([df_SQL,df_Benign])
df.reset_index(inplace = True,drop = True)

In [81]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,478,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,0,1,0,0,0,0,0
1,478,5,1147,5,219,219,27,4294935,31,31,...,0,0,0,0,1,0,0,0,0,0
2,471,5,1147,5,219,219,27,4294945,21,21,...,0,0,0,0,1,0,0,0,0,0
3,461,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,0,1,0,0,0,0,0
4,486,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,0,1,0,0,0,0,0


In [82]:
df.shape

(500354, 45)

In [83]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [84]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,478,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,0,1,0,0,0,0,0
1,478,5,1147,5,219,219,27,4294935,31,31,...,0,0,0,0,1,0,0,0,0,0
2,471,5,1147,5,219,219,27,4294945,21,21,...,0,0,0,0,1,0,0,0,0,0
3,461,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,0,1,0,0,0,0,0
4,486,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,0,1,0,0,0,0,0


In [85]:
Y.head()

,Attack
0,13
1,13
2,13
3,13
4,13


In [86]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [87]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [88]:
np.unique(y_resampled,return_counts = True)

(array([ 0, 13]), array([500000, 500000]))

In [89]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,478,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,1,0,0,0,0,0,13
1,478,5,1147,5,219,219,27,4294935,31,31,...,0,0,0,1,0,0,0,0,0,13
2,471,5,1147,5,219,219,27,4294945,21,21,...,0,0,0,1,0,0,0,0,0,13
3,461,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,1,0,0,0,0,0,13
4,486,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,1,0,0,0,0,0,13


### Undersampling Benign Records Using Numpy Sample

In [90]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [91]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
126609,4675,37,72830,73,223,223,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
163340,2341,25,29933,34,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
192273,89,1,167,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
264250,9634,50,29861,61,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
126826,1472,10,1780,8,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [92]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [93]:
df_SQL = df_new[df_new['Attack']==13]
df_SQL.shape

(500000, 45)

In [94]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [95]:
labels = [1,2,3,4,5,6,7,8,9,10,11,12,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [96]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1035,10,682,10,219,219,27,4294686,281,281,...,0,1,0,0,0,0,0,1,0,0
1,172,4,4101,99,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,172,4,1101,24,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1076,10,682,10,219,219,27,4294748,219,219,...,0,1,0,0,0,0,0,1,0,0
4,1076,10,682,10,219,219,27,4294733,234,234,...,0,1,0,0,0,0,0,1,0,0


In [97]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14]),
 array([  632,  1594,   819, 35000,  2108, 17171, 27167, 35000,    35,
          266,    47, 27031, 15014]))

In [98]:
new_Benign.shape

(35000, 45)

In [99]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
126609,4675,37,72830,73,223,223,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
163340,2341,25,29933,34,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
192273,89,1,167,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
264250,9634,50,29861,61,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
126826,1472,10,1780,8,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [100]:
df.shape

(196884, 45)

In [101]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14]),
 array([35000,   632,  1594,   819, 35000,  2108, 17171, 27167, 35000,
           35,   266,    47, 27031, 15014]))

In [102]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [103]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [104]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [105]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,4675,37,72830,73,223,223,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2341,25,29933,34,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,89,1,167,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,9634,50,29861,61,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1472,10,1780,8,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [106]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [107]:
df_non = df
df_non['Attack'].values[:] = 0

In [108]:
df_non.shape

(490000, 45)

In [109]:
df = pd.concat([df_SQL,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,478,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,1,0,0,0,0,0,13
1,478,5,1147,5,219,219,27,4294935,31,31,...,0,0,0,1,0,0,0,0,0,13
2,471,5,1147,5,219,219,27,4294945,21,21,...,0,0,0,1,0,0,0,0,0,13
3,461,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,1,0,0,0,0,0,13
4,486,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,1,0,0,0,0,0,13


In [110]:
df.shape

(990000, 45)

In [111]:
np.unique(df['Attack'],return_counts = True)

(array([ 0, 13]), array([490000, 500000]))

### XGBoost Model of SQL Injection vs. Others

In [112]:
df.replace({'Attack':13},{'Attack':1},inplace = True)

In [113]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,478,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,1,0,0,0,0,0,1
1,478,5,1147,5,219,219,27,4294935,31,31,...,0,0,0,1,0,0,0,0,0,1
2,471,5,1147,5,219,219,27,4294945,21,21,...,0,0,0,1,0,0,0,0,0,1
3,461,5,1147,5,219,219,27,4294949,17,17,...,0,0,0,1,0,0,0,0,0,1
4,486,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,1,0,0,0,0,0,1


In [114]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [320]:
model_SQL = xgb.XGBClassifier()
# Train the model
model_SQL.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [321]:
y_pred = model_SQL.predict(X_test)

In [322]:
results = classification_report(y_test,y_pred,output_dict = True)

In [323]:
df = pd.DataFrame(results).transpose()

In [324]:
df

,precision,recall,f1-score,support
0,0.999150,0.948707,0.973275,97927.000000
1,0.952170,0.999211,0.975123,100073.000000
accuracy,0.974232,0.974232,0.974232,0.974232
macro avg,0.975660,0.973959,0.974199,198000.000000
weighted avg,0.975405,0.974232,0.974209,198000.000000


### Random Forest of SQL Injection vs. Others

In [115]:
RF_SQL = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
RF_SQL.fit(X_train,y_train)

/tmp/ipykernel_729329/3413761438.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF_SQL.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [116]:
y_pred = RF_SQL.predict(X_test)

In [117]:
results = classification_report(y_test,y_pred,output_dict = True)

In [118]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,0.991000,0.982783,0.986875,97927.000000
1,0.983288,0.991266,0.987261,100073.000000
accuracy,0.987071,0.987071,0.987071,0.987071
macro avg,0.987144,0.987025,0.987068,198000.000000
weighted avg,0.987102,0.987071,0.987070,198000.000000


In [119]:
model_pkl_file = "SQL Injection RF Model.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(RF_SQL, file)

### LightGBM Model of SQL Injection

In [329]:
model_SQL = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_SQL.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_SQL.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013491 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5041
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834


,precision,recall,f1-score,support
0,0.999399,0.916989,0.956422,97927.000000
1,0.924834,0.999460,0.960700,100073.000000
accuracy,0.958672,0.958672,0.958672,0.958672
macro avg,0.962117,0.958225,0.958561,198000.000000
weighted avg,0.961713,0.958672,0.958584,198000.000000


## Brute Force-XSS Model

In [120]:
query_BruteForceXSS = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 3"

In [121]:
cursor.execute(query_BruteForceXSS)

In [122]:
data = cursor.fetchall()

In [123]:
df_BruteForceXSS = pd.DataFrame(data = data, columns = colnames)

In [124]:
df_BruteForceXSS.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,503,5,1147,5,219,219,27,4294940,26,26,...,0,0,0,0,1,0,0,0,0,0
1,479,5,1147,5,219,219,27,4294941,25,25,...,0,0,0,0,1,0,0,0,0,0
2,511,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,0,1,0,0,0,0,0
3,554,5,1147,5,219,219,27,4294939,27,27,...,0,0,0,0,1,0,0,0,0,0
4,501,5,1147,5,219,219,27,4294932,35,35,...,0,0,0,0,1,0,0,0,0,0


In [125]:
df_BruteForceXSS.shape 

(819, 45)

### Fetching Other Types from Database

In [126]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 1000000"

In [127]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [128]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,76,1,160,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,2142,20,10266,21,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,821,14,107772,75,27,27,27,4294920,47,47,...,0,0,0,0,1,0,0,0,0,0
3,70,1,86,1,0,0,0,4294951,16,0,...,0,0,0,1,0,0,0,0,0,1
4,63,1,79,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [129]:
df_Benign.shape

(1000000, 45)

### Concatenate DataFrames For SMOTE

In [130]:
df = pd.concat([df_BruteForceXSS,df_Benign])
df.reset_index(inplace = True,drop = True)

In [131]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,503,5,1147,5,219,219,27,4294940,26,26,...,0,0,0,0,1,0,0,0,0,0
1,479,5,1147,5,219,219,27,4294941,25,25,...,0,0,0,0,1,0,0,0,0,0
2,511,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,0,1,0,0,0,0,0
3,554,5,1147,5,219,219,27,4294939,27,27,...,0,0,0,0,1,0,0,0,0,0
4,501,5,1147,5,219,219,27,4294932,35,35,...,0,0,0,0,1,0,0,0,0,0


In [132]:
df.shape

(1000819, 45)

In [133]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [134]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,503,5,1147,5,219,219,27,4294940,26,26,...,0,0,0,0,1,0,0,0,0,0
1,479,5,1147,5,219,219,27,4294941,25,25,...,0,0,0,0,1,0,0,0,0,0
2,511,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,0,1,0,0,0,0,0
3,554,5,1147,5,219,219,27,4294939,27,27,...,0,0,0,0,1,0,0,0,0,0
4,501,5,1147,5,219,219,27,4294932,35,35,...,0,0,0,0,1,0,0,0,0,0


In [135]:
Y.head()

,Attack
0,3
1,3
2,3
3,3
4,3


In [136]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [137]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [138]:
np.unique(y_resampled,return_counts = True)

(array([0, 3]), array([1000000, 1000000]))

In [139]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,503,5,1147,5,219,219,27,4294940,26,26,...,0,0,0,1,0,0,0,0,0,3
1,479,5,1147,5,219,219,27,4294941,25,25,...,0,0,0,1,0,0,0,0,0,3
2,511,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,1,0,0,0,0,0,3
3,554,5,1147,5,219,219,27,4294939,27,27,...,0,0,0,1,0,0,0,0,0,3
4,501,5,1147,5,219,219,27,4294932,35,35,...,0,0,0,1,0,0,0,0,0,3


### Undersampling Benign Records Using Numpy Sample

In [140]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [141]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
188109,164,3,152,3,23,19,22,0,0,0,...,0,0,1,0,0,0,0,0,1,0
728563,2004,20,5074,16,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
85356,984,11,4727,16,223,218,31,0,0,0,...,0,0,0,0,0,0,0,0,1,0
368521,68,1,165,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
87741,1537,10,7088,9,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [142]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [143]:
df_BruteForceXSS = df_new[df_new['Attack']==3]
df_BruteForceXSS.shape

(1000000, 45)

In [144]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [145]:
labels = [1,2,4,5,6,7,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [146]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,690,8,461,8,223,219,31,4294748,219,219,...,0,1,0,0,0,0,0,1,0,0
1,538,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,690,8,485,8,223,219,31,4294717,250,250,...,0,1,0,0,0,0,0,1,0,0
3,690,8,461,8,223,219,31,4294748,219,219,...,0,1,0,0,0,0,0,1,0,0
4,1028,10,682,10,219,219,27,4294733,234,234,...,0,1,0,0,0,0,0,1,0,0


In [147]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([  632,  1594, 35000,  2108, 17171, 27167, 35000,    35,   266,
           47, 27031,   354, 15014]))

In [148]:
new_Benign.shape

(35000, 45)

In [149]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
188109,164,3,152,3,23,19,22,0,0,0,...,0,0,1,0,0,0,0,0,1,0
728563,2004,20,5074,16,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
85356,984,11,4727,16,223,218,31,0,0,0,...,0,0,0,0,0,0,0,0,1,0
368521,68,1,165,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
87741,1537,10,7088,9,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [150]:
df.shape

(196419, 45)

In [151]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000,   632,  1594, 35000,  2108, 17171, 27167, 35000,    35,
          266,    47, 27031,   354, 15014]))

In [152]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [153]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [154]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [155]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,164,3,152,3,23,19,22,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,2004,20,5074,16,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,984,11,4727,16,223,218,31,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,68,1,165,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,1537,10,7088,9,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [156]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [157]:
df_non = df
df_non['Attack'].values[:] = 0

In [158]:
df_non.shape

(490000, 45)

In [159]:
df = pd.concat([df_BruteForceXSS,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,503,5,1147,5,219,219,27,4294940,26,26,...,0,0,0,1,0,0,0,0,0,3
1,479,5,1147,5,219,219,27,4294941,25,25,...,0,0,0,1,0,0,0,0,0,3
2,511,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,1,0,0,0,0,0,3
3,554,5,1147,5,219,219,27,4294939,27,27,...,0,0,0,1,0,0,0,0,0,3
4,501,5,1147,5,219,219,27,4294932,35,35,...,0,0,0,1,0,0,0,0,0,3


In [160]:
df.shape

(1490000, 45)

In [161]:
np.unique(df['Attack'],return_counts = True)

(array([0, 3]), array([ 490000, 1000000]))

### XGBoost Model of  Brute Force - XSS vs. Others

In [162]:
df.replace({'Attack':3},{'Attack':1},inplace = True)

In [163]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,503,5,1147,5,219,219,27,4294940,26,26,...,0,0,0,1,0,0,0,0,0,1
1,479,5,1147,5,219,219,27,4294941,25,25,...,0,0,0,1,0,0,0,0,0,1
2,511,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,1,0,0,0,0,0,1
3,554,5,1147,5,219,219,27,4294939,27,27,...,0,0,0,1,0,0,0,0,0,1
4,501,5,1147,5,219,219,27,4294932,35,35,...,0,0,0,1,0,0,0,0,0,1


In [164]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [375]:
model_BruteForceXSS = xgb.XGBClassifier()
# Train the model
model_BruteForceXSS.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [376]:
y_pred = model_BruteForceXSS.predict(X_test)

In [377]:
results = classification_report(y_test,y_pred,output_dict = True)

In [378]:
df = pd.DataFrame(results).transpose()

In [379]:
df

,precision,recall,f1-score,support
0,0.998870,0.905787,0.950054,97619.000000
1,0.956096,0.999501,0.977317,200381.000000
accuracy,0.968802,0.968802,0.968802,0.968802
macro avg,0.977483,0.952644,0.963685,298000.000000
weighted avg,0.970108,0.968802,0.968386,298000.000000


### Random Forest of Brute Force - XSS vs. Others

In [165]:
model_BruteForceXSS = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_BruteForceXSS.fit(X_train,y_train)

/tmp/ipykernel_729329/2558936297.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_BruteForceXSS.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [166]:
y_pred = model_BruteForceXSS.predict(X_test)

In [167]:
results = classification_report(y_test,y_pred,output_dict = True)

In [168]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,0.979673,0.965703,0.972638,97619.000000
1,0.983407,0.990239,0.986811,200381.000000
accuracy,0.982201,0.982201,0.982201,0.982201
macro avg,0.981540,0.977971,0.979725,298000.000000
weighted avg,0.982184,0.982201,0.982168,298000.000000


In [170]:
model_pkl_file = "Brute Force-XSS RF Model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_BruteForceXSS, file)

### LightGBM Model of BruteForce-XSS

In [384]:
model_BruteForceXSS = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_BruteForceXSS.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_BruteForceXSS.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()

[LightGBM] [Info] Number of positive: 799619, number of negative: 392381
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.034171 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5628
[LightGBM] [Info] Number of data points in the train set: 1192000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.670821 -> initscore=0.711902
[LightGBM] [Info] Start training from score 0.711902


In [385]:
df

,precision,recall,f1-score,support
0,0.999131,0.871552,0.930991,97619.000000
1,0.941089,0.999631,0.969477,200381.000000
accuracy,0.957674,0.957674,0.957674,0.957674
macro avg,0.970110,0.935591,0.950234,298000.000000
weighted avg,0.960102,0.957674,0.956870,298000.000000


## DDOS Attack - LOIC - UDP

In [171]:
query_DDOS_UDP = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 5"

In [172]:
cursor.execute(query_DDOS_UDP)

In [173]:
data = cursor.fetchall()

In [174]:
df_DDOS_UDP = pd.DataFrame(data = data, columns = colnames)

In [175]:
df_DDOS_UDP.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,5512920,91882,0,0,0,0,0,4190384,104582,0,...,0,0,0,0,0,0,0,0,0,1
1,6110280,101838,0,0,0,0,0,4190243,104723,0,...,0,0,0,0,0,0,0,0,1,0
2,7478400,124640,0,0,0,0,0,4195369,99598,0,...,0,0,0,0,0,0,0,0,0,1
3,7336740,122279,0,0,0,0,0,4195617,99350,0,...,0,0,0,0,0,0,0,0,0,1
4,4974120,82902,0,0,0,0,0,4212636,82331,0,...,0,0,0,0,0,0,0,0,1,0


In [176]:
df_DDOS_UDP.shape

(2108, 45)

### Fetching Other Types from Database

In [177]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 500000"

In [178]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [179]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,656,7,886,7,31,27,30,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1722,21,7522,22,27,27,27,4294952,15,15,...,0,0,0,0,0,0,0,0,0,1
2,2897,25,5267,26,219,219,27,4294936,31,31,...,0,0,0,0,0,0,0,0,0,1
3,1354,16,6978,16,219,219,27,4294951,16,16,...,0,0,0,0,0,0,0,0,0,1
4,4476,26,5619,21,222,222,90,4294811,156,156,...,0,0,0,1,0,0,0,0,0,1


In [180]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [181]:
df = pd.concat([df_DDOS_UDP,df_Benign])
df.reset_index(inplace = True,drop = True)

In [182]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,5512920,91882,0,0,0,0,0,4190384,104582,0,...,0,0,0,0,0,0,0,0,0,1
1,6110280,101838,0,0,0,0,0,4190243,104723,0,...,0,0,0,0,0,0,0,0,1,0
2,7478400,124640,0,0,0,0,0,4195369,99598,0,...,0,0,0,0,0,0,0,0,0,1
3,7336740,122279,0,0,0,0,0,4195617,99350,0,...,0,0,0,0,0,0,0,0,0,1
4,4974120,82902,0,0,0,0,0,4212636,82331,0,...,0,0,0,0,0,0,0,0,1,0


In [183]:
df.shape

(502108, 45)

In [184]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [185]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,5512920,91882,0,0,0,0,0,4190384,104582,0,...,0,0,0,0,0,0,0,0,0,1
1,6110280,101838,0,0,0,0,0,4190243,104723,0,...,0,0,0,0,0,0,0,0,1,0
2,7478400,124640,0,0,0,0,0,4195369,99598,0,...,0,0,0,0,0,0,0,0,0,1
3,7336740,122279,0,0,0,0,0,4195617,99350,0,...,0,0,0,0,0,0,0,0,0,1
4,4974120,82902,0,0,0,0,0,4212636,82331,0,...,0,0,0,0,0,0,0,0,1,0


In [186]:
Y.head()

,Attack
0,5
1,5
2,5
3,5
4,5


In [187]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [188]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [189]:
np.unique(y_resampled,return_counts = True)

(array([0, 5]), array([500000, 500000]))

In [190]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,5512920,91882,0,0,0,0,0,4190384,104582,0,...,0,0,0,0,0,0,0,0,1,5
1,6110280,101838,0,0,0,0,0,4190243,104723,0,...,0,0,0,0,0,0,0,1,0,5
2,7478400,124640,0,0,0,0,0,4195369,99598,0,...,0,0,0,0,0,0,0,0,1,5
3,7336740,122279,0,0,0,0,0,4195617,99350,0,...,0,0,0,0,0,0,0,0,1,5
4,4974120,82902,0,0,0,0,0,4212636,82331,0,...,0,0,0,0,0,0,0,1,0,5


### Undersampling Benign Records Using Numpy Sample

In [191]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [192]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
128363,2031,20,11678,21,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
165094,3395,15,7571,20,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
194027,2473,27,45947,43,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
266004,83,1,254,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
128580,67,1,228,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [193]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [194]:
df_DDOS_UDP = df_new[df_new['Attack']==5]
df_DDOS_UDP.shape

(500000, 45)

In [195]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [196]:
labels = [1,2,3,4,6,7,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [197]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1115,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
1,1076,10,682,10,219,219,27,4294686,281,281,...,0,1,0,0,0,0,0,1,0,0
2,690,8,461,8,223,219,31,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
3,987,10,682,10,219,219,27,4294732,235,235,...,0,1,0,0,0,0,0,1,0,0
4,710,9,1202,23,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0


In [198]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([  632,  1594,   819, 35000, 17171, 27167, 35000,    35,   266,
           47, 27031,   354, 15014]))

In [199]:
new_Benign.shape

(35000, 45)

In [200]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
128363,2031,20,11678,21,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
165094,3395,15,7571,20,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
194027,2473,27,45947,43,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
266004,83,1,254,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
128580,67,1,228,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [201]:
df.shape

(195130, 45)

In [202]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000,   632,  1594,   819, 35000, 17171, 27167, 35000,    35,
          266,    47, 27031,   354, 15014]))

In [203]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [204]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [205]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [206]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,2031,20,11678,21,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3395,15,7571,20,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2473,27,45947,43,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,83,1,254,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,67,1,228,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [207]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [208]:
df_non = df
df_non['Attack'].values[:] = 0

In [209]:
df_non.shape

(490000, 45)

In [210]:
df = pd.concat([df_DDOS_UDP,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,5512920,91882,0,0,0,0,0,4190384,104582,0,...,0,0,0,0,0,0,0,0,1,5
1,6110280,101838,0,0,0,0,0,4190243,104723,0,...,0,0,0,0,0,0,0,1,0,5
2,7478400,124640,0,0,0,0,0,4195369,99598,0,...,0,0,0,0,0,0,0,0,1,5
3,7336740,122279,0,0,0,0,0,4195617,99350,0,...,0,0,0,0,0,0,0,0,1,5
4,4974120,82902,0,0,0,0,0,4212636,82331,0,...,0,0,0,0,0,0,0,1,0,5


In [211]:
df.shape

(990000, 45)

In [212]:
np.unique(df['Attack'],return_counts = True)

(array([0, 5]), array([490000, 500000]))

### XGBoost Model of DDOS Attacks - LOIC - UDP vs. Others

In [213]:
df.replace({'Attack':5},{'Attack':1},inplace = True)

In [214]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,5512920,91882,0,0,0,0,0,4190384,104582,0,...,0,0,0,0,0,0,0,0,1,1
1,6110280,101838,0,0,0,0,0,4190243,104723,0,...,0,0,0,0,0,0,0,1,0,1
2,7478400,124640,0,0,0,0,0,4195369,99598,0,...,0,0,0,0,0,0,0,0,1,1
3,7336740,122279,0,0,0,0,0,4195617,99350,0,...,0,0,0,0,0,0,0,0,1,1
4,4974120,82902,0,0,0,0,0,4212636,82331,0,...,0,0,0,0,0,0,0,1,0,1


In [215]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [431]:
model_DDOS_UDP = xgb.XGBClassifier()
# Train the model
model_DDOS_UDP.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [432]:
y_pred = model_DDOS_UDP.predict(X_test)

In [433]:
results = classification_report(y_test,y_pred,output_dict = True)

In [434]:
df = pd.DataFrame(results).transpose()

In [435]:
df

,precision,recall,f1-score,support
0,0.999959,0.999816,0.999888,97927.000000
1,0.999820,0.999960,0.999890,100073.000000
accuracy,0.999889,0.999889,0.999889,0.999889
macro avg,0.999890,0.999888,0.999889,198000.000000
weighted avg,0.999889,0.999889,0.999889,198000.000000


### Random Forest of DDOS Attacks - LOIC - UDP vs. Others

In [216]:
model_DDOS_UDP = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_DDOS_UDP.fit(X_train,y_train)

/tmp/ipykernel_729329/4185831354.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_DDOS_UDP.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [217]:
y_pred = model_DDOS_UDP.predict(X_test)

In [218]:
results = classification_report(y_test,y_pred,output_dict = True)

In [219]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,0.999939,0.999888,0.999913,97927.000000
1,0.999890,0.999940,0.999915,100073.000000
accuracy,0.999914,0.999914,0.999914,0.999914
macro avg,0.999914,0.999914,0.999914,198000.000000
weighted avg,0.999914,0.999914,0.999914,198000.000000


In [319]:
model_pkl_file = "DDoS Atacks-LOIC-UDP RF Model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_DDOS_UDP, file)

### LightGBM Model of DDoS Attacks-LOIC-UDP

In [440]:
model_DDOS_UDP = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_DDOS_UDP.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_DDOS_UDP.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.019630 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5072
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834


,precision,recall,f1-score,support
0,0.999939,0.999816,0.999877,97927.000000
1,0.999820,0.999940,0.999880,100073.000000
accuracy,0.999879,0.999879,0.999879,0.999879
macro avg,0.999879,0.999878,0.999879,198000.000000
weighted avg,0.999879,0.999879,0.999879,198000.000000


## Brute Force Web Model

In [221]:
query_BruteForceWeb = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 2"

In [222]:
cursor.execute(query_BruteForceWeb)

In [223]:
data = cursor.fetchall()

In [224]:
df_BruteForceWeb = pd.DataFrame(data = data, columns = colnames)

In [225]:
df_BruteForceWeb.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,511,5,1147,5,219,219,27,4294946,21,21,...,0,0,0,0,1,0,0,0,0,0
1,553,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,0,1,0,0,0,0,0
2,61207,155,76705,105,219,219,27,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,577,5,1147,5,219,219,27,4294947,19,19,...,0,0,0,0,1,0,0,0,0,0
4,528,5,1147,5,219,219,27,4294952,15,15,...,0,0,0,0,1,0,0,0,0,0


In [226]:
df_BruteForceWeb.shape

(1594, 45)

### Fetching Other Types from Database

In [227]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 1500000"

In [228]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [229]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1932,14,1991,10,223,219,30,4294952,0,15,...,0,0,0,1,0,0,0,0,0,1
1,11136,119,17307,157,219,219,27,4294951,16,16,...,0,0,0,0,0,0,0,0,0,1
2,687,9,5459,11,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,585,5,384,5,30,26,30,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,87,1,215,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [230]:
df_Benign.shape

(1500000, 45)

### Concatenate DataFrames For SMOTE

In [231]:
df = pd.concat([df_BruteForceWeb,df_Benign])
df.reset_index(inplace = True,drop = True)

In [232]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,511,5,1147,5,219,219,27,4294946,21,21,...,0,0,0,0,1,0,0,0,0,0
1,553,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,0,1,0,0,0,0,0
2,61207,155,76705,105,219,219,27,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,577,5,1147,5,219,219,27,4294947,19,19,...,0,0,0,0,1,0,0,0,0,0
4,528,5,1147,5,219,219,27,4294952,15,15,...,0,0,0,0,1,0,0,0,0,0


In [233]:
df.shape

(1501594, 45)

In [234]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [235]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,511,5,1147,5,219,219,27,4294946,21,21,...,0,0,0,0,1,0,0,0,0,0
1,553,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,0,1,0,0,0,0,0
2,61207,155,76705,105,219,219,27,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,577,5,1147,5,219,219,27,4294947,19,19,...,0,0,0,0,1,0,0,0,0,0
4,528,5,1147,5,219,219,27,4294952,15,15,...,0,0,0,0,1,0,0,0,0,0


In [236]:
Y.head()

,Attack
0,2
1,2
2,2
3,2
4,2


In [237]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [238]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [239]:
np.unique(y_resampled,return_counts = True)

(array([0, 2]), array([1500000, 1500000]))

In [240]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,511,5,1147,5,219,219,27,4294946,21,21,...,0,0,0,1,0,0,0,0,0,2
1,553,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,1,0,0,0,0,0,2
2,61207,155,76705,105,219,219,27,0,0,0,...,0,0,0,1,0,0,0,0,0,2
3,577,5,1147,5,219,219,27,4294947,19,19,...,0,0,0,1,0,0,0,0,0,2
4,528,5,1147,5,219,219,27,4294952,15,15,...,0,0,0,1,0,0,0,0,0,2


### Undersampling Benign Records Using Numpy Sample

In [241]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [242]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
401004,22623,104,259796,209,223,223,30,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0
1119335,3625,29,12272,27,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1110980,2199,23,5909,22,223,223,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1495389,132,3,132,3,23,19,22,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1294135,1559,9,2043,10,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [243]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [244]:
df_BruteForceWeb = df_new[df_new['Attack']==2]
df_BruteForceWeb.shape

(1500000, 45)

In [245]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [246]:
labels = [1,3,4,5,6,7,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [247]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,244,5,821,17,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,172,4,421,7,219,211,27,4294951,16,16,...,0,1,0,0,0,0,0,1,0,0
2,1076,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
3,1076,10,682,10,219,219,27,4294733,234,234,...,0,1,0,0,0,0,0,1,0,0
4,1076,10,693,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0


In [248]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([  632,   819, 35000,  2108, 17171, 27167, 35000,    35,   266,
           47, 27031,   354, 15014]))

In [249]:
new_Benign.shape

(35000, 45)

In [250]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
401004,22623,104,259796,209,223,223,30,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0
1119335,3625,29,12272,27,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1110980,2199,23,5909,22,223,223,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1495389,132,3,132,3,23,19,22,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1294135,1559,9,2043,10,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [251]:
df.shape

(195644, 45)

In [252]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000,   632,   819, 35000,  2108, 17171, 27167, 35000,    35,
          266,    47, 27031,   354, 15014]))

In [253]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [254]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [255]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [256]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,22623,104,259796,209,223,223,30,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0
1,3625,29,12272,27,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2199,23,5909,22,223,223,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,132,3,132,3,23,19,22,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,1559,9,2043,10,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [257]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [258]:
df_non = df
df_non['Attack'].values[:] = 0

In [259]:
df_non.shape

(490000, 45)

In [260]:
df = pd.concat([df_BruteForceWeb,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,511,5,1147,5,219,219,27,4294946,21,21,...,0,0,0,1,0,0,0,0,0,2
1,553,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,1,0,0,0,0,0,2
2,61207,155,76705,105,219,219,27,0,0,0,...,0,0,0,1,0,0,0,0,0,2
3,577,5,1147,5,219,219,27,4294947,19,19,...,0,0,0,1,0,0,0,0,0,2
4,528,5,1147,5,219,219,27,4294952,15,15,...,0,0,0,1,0,0,0,0,0,2


In [261]:
df.shape

(1990000, 45)

In [262]:
np.unique(df['Attack'],return_counts = True)

(array([0, 2]), array([ 490000, 1500000]))

### XGBoost Model of Brute Force Web vs. Others

In [263]:
df.replace({'Attack':2},{'Attack':1},inplace = True)

In [264]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,511,5,1147,5,219,219,27,4294946,21,21,...,0,0,0,1,0,0,0,0,0,1
1,553,5,1147,5,219,219,27,4294964,3,3,...,0,0,0,1,0,0,0,0,0,1
2,61207,155,76705,105,219,219,27,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,577,5,1147,5,219,219,27,4294947,19,19,...,0,0,0,1,0,0,0,0,0,1
4,528,5,1147,5,219,219,27,4294952,15,15,...,0,0,0,1,0,0,0,0,0,1


In [265]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [486]:
model_BruteForce_Web = xgb.XGBClassifier()
# Train the model
model_BruteForce_Web.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [487]:
y_pred = model_BruteForce_Web.predict(X_test)

In [488]:
results = classification_report(y_test,y_pred,output_dict = True)

In [489]:
df = pd.DataFrame(results).transpose()

In [490]:
df

,precision,recall,f1-score,support
0,0.998808,0.886782,0.939467,98306.000000
1,0.964180,0.999653,0.981596,299694.000000
accuracy,0.971774,0.971774,0.971774,0.971774
macro avg,0.981494,0.943218,0.960532,398000.000000
weighted avg,0.972733,0.971774,0.971190,398000.000000


### Random Forest of Brute Force Web vs. Others

In [266]:
model_BruteForce_Web = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_BruteForce_Web.fit(X_train,y_train)

/tmp/ipykernel_729329/393213215.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_BruteForce_Web.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [267]:
y_pred = model_BruteForce_Web.predict(X_test)

In [268]:
results = classification_report(y_test,y_pred,output_dict = True)

In [269]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,0.972469,0.952546,0.962405,98306.000000
1,0.984538,0.991154,0.987835,299694.000000
accuracy,0.981618,0.981618,0.981618,0.981618
macro avg,0.978504,0.971850,0.975120,398000.000000
weighted avg,0.981557,0.981618,0.981554,398000.000000


In [318]:
model_pkl_file = "Brute Force-Web RF Model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_BruteForce_Web, file)

### LightGBM Model of BruteForce-Web

In [495]:
model_BruteForce_Web = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_BruteForce_Web.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_BruteForce_Web.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 1200306, number of negative: 391694
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042771 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5420
[LightGBM] [Info] Number of data points in the train set: 1592000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.753961 -> initscore=1.119851
[LightGBM] [Info] Start training from score 1.119851


,precision,recall,f1-score,support
0,0.999409,0.843417,0.914811,98306.000000
1,0.951139,0.999836,0.974880,299694.000000
accuracy,0.961201,0.961201,0.961201,0.961201
macro avg,0.975274,0.921627,0.944846,398000.000000
weighted avg,0.963062,0.961201,0.960043,398000.000000


## SSH - Brute Force Model

In [271]:
query_SSH_BruteForce = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 14"

In [272]:
cursor.execute(query_SSH_BruteForce)

In [273]:
data = cursor.fetchall()

In [274]:
df_SSH_BruteForce = pd.DataFrame(data = data, columns = colnames)

In [275]:
df_SSH_BruteForce.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,3164,23,3765,21,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,21940,161,26823,156,27,27,27,4292983,1984,1984,...,0,0,0,0,0,0,1,0,0,0
2,3116,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3168,24,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,3132,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [276]:
df_SSH_BruteForce.shape

(15014, 45)

### Fetching Other Types from Database

In [277]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 500000"

In [278]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [279]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,63,1,127,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,40,1,0,0,2,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1471,21,6476,23,219,219,219,4294951,16,16,...,0,0,0,0,0,0,0,0,0,1
3,1355,16,6087,16,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,64,1,153,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [280]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [281]:
df = pd.concat([df_SSH_BruteForce,df_Benign])
df.reset_index(inplace = True,drop = True)

In [282]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,3164,23,3765,21,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,21940,161,26823,156,27,27,27,4292983,1984,1984,...,0,0,0,0,0,0,1,0,0,0
2,3116,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3168,24,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,3132,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [283]:
df.shape

(515014, 45)

In [284]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [285]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,3164,23,3765,21,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,21940,161,26823,156,27,27,27,4292983,1984,1984,...,0,0,0,0,0,0,1,0,0,0
2,3116,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,3168,24,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,3132,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [286]:
Y.head()

,Attack
0,14
1,14
2,14
3,14
4,14


In [287]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [288]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [289]:
np.unique(y_resampled,return_counts = True)

(array([ 0, 14]), array([500000, 500000]))

In [290]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,3164,23,3765,21,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,14
1,21940,161,26823,156,27,27,27,4292983,1984,1984,...,0,0,0,0,0,1,0,0,0,14
2,3116,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,14
3,3168,24,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,14
4,3132,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,14


### Undersampling Benign Records Using Numpy Sample

In [291]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [292]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
141269,629,6,428,6,30,26,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0
178000,72,1,136,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
206933,2018,9,4403,9,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
278910,11758,82,17597,84,222,222,26,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0
141486,2154,20,4816,17,27,27,27,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0


In [293]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [294]:
df_SSH_BruteForce = df_new[df_new['Attack']==14]
df_SSH_BruteForce.shape

(500000, 45)

In [295]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [296]:
labels = [1,2,3,4,5,6,7,8,9,10,11,12,13]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [297]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1076,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
1,172,4,501,9,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,1648,5,821,17,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,608,5,2621,62,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,172,4,701,14,219,211,27,4294951,16,16,...,0,1,0,0,0,0,0,1,0,0


In [298]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
 array([  632,  1594,   819, 35000,  2108, 17171, 27167, 35000,    35,
          266,    47, 27031,   354]))

In [299]:
new_Benign.shape

(35000, 45)

In [300]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
141269,629,6,428,6,30,26,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0
178000,72,1,136,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
206933,2018,9,4403,9,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
278910,11758,82,17597,84,222,222,26,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0
141486,2154,20,4816,17,27,27,27,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0


In [301]:
df.shape

(182224, 45)

In [302]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
 array([35000,   632,  1594,   819, 35000,  2108, 17171, 27167, 35000,
           35,   266,    47, 27031,   354]))

In [303]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [304]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [305]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [306]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,629,6,428,6,30,26,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,72,1,136,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,2018,9,4403,9,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,11758,82,17597,84,222,222,26,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0
4,2154,20,4816,17,27,27,27,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0


In [307]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [308]:
df_non = df
df_non['Attack'].values[:] = 0

In [309]:
df_non.shape

(490000, 45)

In [310]:
df = pd.concat([df_SSH_BruteForce,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,3164,23,3765,21,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,14
1,21940,161,26823,156,27,27,27,4292983,1984,1984,...,0,0,0,0,0,1,0,0,0,14
2,3116,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,14
3,3168,24,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,14
4,3132,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,14


In [311]:
df.shape

(990000, 45)

In [312]:
np.unique(df['Attack'],return_counts = True)

(array([ 0, 14]), array([490000, 500000]))

### XGBoost Model of SSH - Brute Force vs. Others

In [538]:
df.replace({'Attack':14},{'Attack':1},inplace = True)

In [539]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,3164,23,3765,21,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,21940,161,26823,156,27,27,27,4292983,1984,1984,...,0,0,0,0,0,1,0,0,0,1
2,3116,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,3168,24,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,3132,23,3869,23,27,27,27,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [540]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [541]:
model_SSH_BruteForce = xgb.XGBClassifier()
# Train the model
model_SSH_BruteForce.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [542]:
y_pred = model_SSH_BruteForce.predict(X_test)

In [543]:
results = classification_report(y_test,y_pred,output_dict = True)

In [544]:
df = pd.DataFrame(results).transpose()

In [545]:
df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


### Random Forest of SSH - Brute Force vs. Others

In [313]:
model_SSH_BruteForce = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_SSH_BruteForce.fit(X_train,y_train)

/tmp/ipykernel_729329/3274350401.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_SSH_BruteForce.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [314]:
y_pred = model_SSH_BruteForce.predict(X_test)

In [315]:
results = classification_report(y_test,y_pred,output_dict = True)

In [316]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,0.972469,0.952546,0.962405,98306.000000
1,0.984538,0.991154,0.987835,299694.000000
accuracy,0.981618,0.981618,0.981618,0.981618
macro avg,0.978504,0.971850,0.975120,398000.000000
weighted avg,0.981557,0.981618,0.981554,398000.000000


In [317]:
model_pkl_file = "SSH-Brute Force RF Model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_SSH_BruteForce, file)

### LightGBM Model of SSH-BruteForce

In [550]:
model_SSH_BruteForce = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_SSH_BruteForce.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_SSH_BruteForce.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004892 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4976
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


## Benign Model

In [8]:
query_benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 500000"

In [9]:
cursor.execute(query_benign)

In [10]:
data = cursor.fetchall()

In [11]:
df_benign = pd.DataFrame(data = data, columns = colnames)

In [12]:
df_benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,762,6,569,6,30,26,30,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1544,10,1873,7,30,30,26,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1126,13,5094,12,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,693,9,4099,9,219,219,91,4294951,16,16,...,0,0,0,0,0,0,0,0,0,1
4,694,9,4408,11,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
df_benign.shape 

(500000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [14]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [15]:
labels = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [16]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,690,8,461,8,223,219,31,4294733,234,234,...,0,1,0,0,0,0,0,1,0,0
1,1115,10,682,10,219,219,27,4294732,235,235,...,0,1,0,0,0,0,0,1,0,0
2,1076,10,682,10,219,219,27,4294748,219,219,...,0,1,0,0,0,0,0,1,0,0
3,172,4,4621,112,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,710,9,962,17,219,219,27,4294716,251,251,...,0,1,0,0,0,0,0,1,0,0


In [17]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([  632,  1594,   819, 35000,  2108, 17171, 27167, 35000,    35,
          266,    47, 27031,   354, 15014]))

In [18]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [19]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [20]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fa09866dca0>
Traceback (most recent call last):
  File "/home/sharaf/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/sharaf/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/sharaf/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/sharaf/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.mat

Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.match_module_callback at 0x7fa002e6d670>
Traceback (most recent call last):
  File "/home/sharaf/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 400, in match_module_callback
    self._make_module_from_path(filepath)
  File "/home/sharaf/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 515, in _make_module_from_path
    module = module_class(filepath, prefix, user_api, internal_api)
  File "/home/sharaf/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 606, in __init__
    self.version = self.get_version()
  File "/home/sharaf/anaconda3/lib/python3.9/site-packages/threadpoolctl.py", line 646, in get_version
    config = get_config().split()
AttributeError: 'NoneType' object has no attribute 'split'
Exception ignored on calling ctypes callback function: <function _ThreadpoolInfo._find_modules_with_dl_iterate_phdr.<locals>.mat

In [21]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,690,8,461,8,223,219,31,4294733,234,234,...,1,0,0,0,0,0,1,0,0,1
1,1115,10,682,10,219,219,27,4294732,235,235,...,1,0,0,0,0,0,1,0,0,1
2,1076,10,682,10,219,219,27,4294748,219,219,...,1,0,0,0,0,0,1,0,0,1
3,172,4,4621,112,219,211,27,0,0,0,...,1,0,0,0,0,0,1,0,0,1
4,710,9,962,17,219,219,27,4294716,251,251,...,1,0,0,0,0,0,1,0,0,1


In [22]:
np.unique(df['Attack'],return_counts = True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [23]:
df_non = df
df_non['Attack'].values[:] = 0
df_benign.replace({'Attack':0},{'Attack':1},inplace = True)

In [24]:
df_non.shape

(490000, 45)

In [25]:
df = pd.concat([df_benign,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,762,6,569,6,30,26,30,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1544,10,1873,7,30,30,26,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1126,13,5094,12,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,693,9,4099,9,219,219,91,4294951,16,16,...,0,0,0,0,0,0,0,0,0,1
4,694,9,4408,11,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [26]:
df.shape

(990000, 45)

In [27]:
np.unique(df['Attack'],return_counts = True)

(array([0, 1]), array([490000, 500000]))

### XGBoost Model of Benign vs. Others

In [28]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [29]:
model_benign = xgb.XGBClassifier()
# Train the model
model_benign.fit(X_train,y_train)

/home/sharaf/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sharaf/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sharaf/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dt

[17:49:58] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=20, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
y_pred = model_benign.predict(X_test)

In [31]:
results = classification_report(y_test,y_pred,output_dict = True)

In [32]:
df = pd.DataFrame(results).transpose()

In [33]:
df

,precision,recall,f1-score,support
0,0.992613,0.956345,0.974141,97927.000000
1,0.958756,0.993035,0.975594,100073.000000
accuracy,0.974889,0.974889,0.974889,0.974889
macro avg,0.975684,0.974690,0.974868,198000.000000
weighted avg,0.975501,0.974889,0.974876,198000.000000


In [34]:
model_pkl_file = "Benign XGB Model.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_benign, file)

### Random Forest of Benign vs. Others

In [577]:
model_benign = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_benign.fit(X_train,y_train)

/tmp/ipykernel_959012/2774199211.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_benign.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [578]:
y_pred = model_benign.predict(X_test)

In [579]:
results = classification_report(y_test,y_pred,output_dict = True)

In [580]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,0.977744,0.964096,0.970872,97927.000000
1,0.965339,0.978526,0.971888,100073.000000
accuracy,0.971389,0.971389,0.971389,0.971389
macro avg,0.971542,0.971311,0.971380,198000.000000
weighted avg,0.971475,0.971389,0.971385,198000.000000


### LightGBM Model of Benign

In [581]:
model_benign = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_benign.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_benign.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011329 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5618
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834


,precision,recall,f1-score,support
0,0.995397,0.949646,0.971984,97927.000000
1,0.952847,0.995703,0.973804,100073.000000
accuracy,0.972924,0.972924,0.972924,0.972924
macro avg,0.974122,0.972675,0.972894,198000.000000
weighted avg,0.973892,0.972924,0.972904,198000.000000


## Infiltration Model

In [347]:
query_infiltration = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 12"

In [348]:
cursor.execute(query_infiltration)

In [349]:
data = cursor.fetchall()

In [350]:
df_infiltration = pd.DataFrame(data = data, columns = colnames)

In [351]:
df_infiltration.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,68,1,167,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1460,8,1873,7,222,222,90,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,75,1,181,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,64,1,96,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,70,1,86,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [352]:
df_infiltration.shape

(27031, 45)

### Fetching Other Types from Database

In [353]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 500000"

In [354]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [355]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1960,23,38959,37,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2725,25,8367,22,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,66,1,94,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,70,1,86,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,50,1,119,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [356]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [357]:
df = pd.concat([df_infiltration,df_Benign])
df.reset_index(inplace = True,drop = True)

In [358]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,68,1,167,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1460,8,1873,7,222,222,90,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,75,1,181,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,64,1,96,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,70,1,86,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [359]:
df.shape

(527031, 45)

In [360]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [361]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,68,1,167,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1460,8,1873,7,222,222,90,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,75,1,181,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,64,1,96,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,70,1,86,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [362]:
Y.head()

,Attack
0,12
1,12
2,12
3,12
4,12


In [363]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [364]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [365]:
np.unique(y_resampled,return_counts = True)

(array([ 0, 12]), array([500000, 500000]))

In [366]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,68,1,167,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,12
1,1460,8,1873,7,222,222,90,0,0,0,...,0,0,1,0,0,0,0,0,1,12
2,75,1,181,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,12
3,64,1,96,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,12
4,70,1,86,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,12


### Undersampling Benign Records Using Numpy Sample

In [367]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [368]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
153286,67,1,83,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
190017,1984,25,20332,29,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
218950,1556,10,1869,7,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0
290927,707,6,528,5,30,26,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0
153503,2010,17,5176,19,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [369]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [370]:
df_infiltration = df_new[df_new['Attack']==12]
df_infiltration.shape

(500000, 45)

In [371]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [372]:
labels = [1,2,3,4,5,6,7,8,9,10,11,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [373]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,172,4,821,17,219,211,27,4294951,16,16,...,0,1,0,0,0,0,0,1,0,0
1,690,8,461,8,223,219,31,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
2,1076,10,693,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
3,1115,10,682,10,219,219,27,4294732,235,235,...,0,1,0,0,0,0,0,1,0,0
4,172,4,941,20,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [374]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14]),
 array([  632,  1594,   819, 35000,  2108, 17171, 27167, 35000,    35,
          266,    47,   354, 15014]))

In [375]:
new_Benign.shape

(35000, 45)

In [376]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
153286,67,1,83,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
190017,1984,25,20332,29,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
218950,1556,10,1869,7,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0
290927,707,6,528,5,30,26,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0
153503,2010,17,5176,19,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [377]:
df.shape

(170207, 45)

In [378]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14]),
 array([35000,   632,  1594,   819, 35000,  2108, 17171, 27167, 35000,
           35,   266,    47,   354, 15014]))

In [379]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [380]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [381]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [382]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,67,1,83,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,1984,25,20332,29,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1556,10,1869,7,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0
3,707,6,528,5,30,26,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,2010,17,5176,19,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [383]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [384]:
df_non = df
df_non['Attack'].values[:] = 0

In [385]:
df_non.shape

(490000, 45)

In [386]:
df = pd.concat([df_infiltration,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,68,1,167,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,12
1,1460,8,1873,7,222,222,90,0,0,0,...,0,0,1,0,0,0,0,0,1,12
2,75,1,181,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,12
3,64,1,96,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,12
4,70,1,86,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,12


In [387]:
df.shape

(990000, 45)

In [388]:
np.unique(df['Attack'],return_counts = True)

(array([ 0, 12]), array([490000, 500000]))

### XGBoost Model of Infiltration vs. Others

In [389]:
df.replace({'Attack':12},{'Attack':1},inplace = True)

In [390]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,68,1,167,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
1,1460,8,1873,7,222,222,90,0,0,0,...,0,0,1,0,0,0,0,0,1,1
2,75,1,181,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
3,64,1,96,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1
4,70,1,86,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,1


In [391]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [627]:
model_infiltration = xgb.XGBClassifier()
# Train the model
model_infiltration.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [628]:
y_pred = model_infiltration.predict(X_test)

In [629]:
results = classification_report(y_test,y_pred,output_dict = True)

In [630]:
df = pd.DataFrame(results).transpose()

In [631]:
df

,precision,recall,f1-score,support
0,0.996404,0.976156,0.986176,97927.000000
1,0.977122,0.996553,0.986742,100073.000000
accuracy,0.986465,0.986465,0.986465,0.986465
macro avg,0.986763,0.986354,0.986459,198000.000000
weighted avg,0.986658,0.986465,0.986462,198000.000000


### Random Forest of Infiltration vs. Others

In [392]:
model_infiltration = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_infiltration.fit(X_train,y_train)

/tmp/ipykernel_729329/3934741868.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_infiltration.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [393]:
y_pred = model_infiltration.predict(X_test)

In [394]:
results = classification_report(y_test,y_pred,output_dict = True)

In [395]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,0.996019,0.983518,0.989729,97927.000000
1,0.984067,0.996153,0.990073,100073.000000
accuracy,0.989904,0.989904,0.989904,0.989904
macro avg,0.990043,0.989836,0.989901,198000.000000
weighted avg,0.989978,0.989904,0.989903,198000.000000


In [396]:
model_pkl_file = "Infiltration RF Model.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_infiltration, file)

### LightGBM Model of Infiltration

In [636]:
model_infiltration = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_infiltration.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_infiltration.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015763 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6069
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834


,precision,recall,f1-score,support
0,0.997168,0.967384,0.982050,97927.00000
1,0.968990,0.997312,0.982947,100073.00000
accuracy,0.982510,0.982510,0.982510,0.98251
macro avg,0.983079,0.982348,0.982499,198000.00000
weighted avg,0.982926,0.982510,0.982503,198000.00000


## FTP-Brute Force Model

In [397]:
query_FTP = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 11"

In [398]:
cursor.execute(query_FTP)

In [399]:
data = cursor.fetchall()

In [400]:
df_FTP = pd.DataFrame(data = data, columns = colnames)

In [401]:
df_FTP.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,780,13,520,13,22,2,20,4294763,204,204,...,0,0,0,0,0,0,0,0,0,1
1,780,13,520,13,22,2,20,4294779,188,188,...,0,0,0,0,0,0,0,0,0,1
2,840,14,560,14,22,2,20,4294732,235,235,...,0,0,0,0,0,0,0,0,0,1
3,840,14,560,14,22,2,20,4294748,219,219,...,0,0,0,0,0,0,0,0,0,1
4,840,14,560,14,22,2,20,4294717,250,250,...,0,0,0,0,0,0,0,0,0,1


In [402]:
df_FTP.shape

(47, 45)

### Fetching Other Types from Database

In [403]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 500000"

In [404]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [405]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1150,17,1657,15,27,27,27,4294935,32,32,...,0,0,0,0,1,0,0,0,0,0
1,1481,15,4723,15,219,219,27,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2879,28,8060,30,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,1376,18,30002,34,219,219,27,4294951,16,16,...,0,0,0,0,1,0,0,0,0,0
4,1757,11,5005,15,27,27,27,4294951,16,16,...,0,0,0,0,0,0,0,0,0,1


In [406]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [407]:
df = pd.concat([df_FTP,df_Benign])
df.reset_index(inplace = True,drop = True)

In [408]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,780,13,520,13,22,2,20,4294763,204,204,...,0,0,0,0,0,0,0,0,0,1
1,780,13,520,13,22,2,20,4294779,188,188,...,0,0,0,0,0,0,0,0,0,1
2,840,14,560,14,22,2,20,4294732,235,235,...,0,0,0,0,0,0,0,0,0,1
3,840,14,560,14,22,2,20,4294748,219,219,...,0,0,0,0,0,0,0,0,0,1
4,840,14,560,14,22,2,20,4294717,250,250,...,0,0,0,0,0,0,0,0,0,1


In [409]:
df.shape

(500047, 45)

In [410]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [411]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,780,13,520,13,22,2,20,4294763,204,204,...,0,0,0,0,0,0,0,0,0,1
1,780,13,520,13,22,2,20,4294779,188,188,...,0,0,0,0,0,0,0,0,0,1
2,840,14,560,14,22,2,20,4294732,235,235,...,0,0,0,0,0,0,0,0,0,1
3,840,14,560,14,22,2,20,4294748,219,219,...,0,0,0,0,0,0,0,0,0,1
4,840,14,560,14,22,2,20,4294717,250,250,...,0,0,0,0,0,0,0,0,0,1


In [412]:
Y.head()

,Attack
0,11
1,11
2,11
3,11
4,11


In [413]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [414]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [415]:
np.unique(y_resampled,return_counts = True)

(array([ 0, 11]), array([500000, 500000]))

In [416]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,780,13,520,13,22,2,20,4294763,204,204,...,0,0,0,0,0,0,0,0,1,11
1,780,13,520,13,22,2,20,4294779,188,188,...,0,0,0,0,0,0,0,0,1,11
2,840,14,560,14,22,2,20,4294732,235,235,...,0,0,0,0,0,0,0,0,1,11
3,840,14,560,14,22,2,20,4294748,219,219,...,0,0,0,0,0,0,0,0,1,11
4,840,14,560,14,22,2,20,4294717,250,250,...,0,0,0,0,0,0,0,0,1,11


### Undersampling Benign Records Using Numpy Sample

In [417]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [418]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
126302,1744,20,26454,32,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
163033,126,2,190,2,0,0,0,4294952,15,0,...,0,0,1,0,0,0,0,0,1,0
191966,80,2,0,0,2,2,0,4294733,234,0,...,0,0,0,0,0,1,0,0,0,0
263943,655,9,3428,7,27,27,27,0,0,0,...,0,0,0,1,0,0,0,0,0,0
126519,581,5,420,6,30,26,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [419]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [420]:
df_FTP = df_new[df_new['Attack']==11]
df_FTP.shape

(500000, 45)

In [421]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [422]:
labels = [1,2,3,4,5,6,7,8,9,10,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [423]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1076,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
1,1035,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
2,1076,10,682,10,219,219,27,4294702,265,265,...,0,1,0,0,0,0,0,1,0,0
3,172,4,501,9,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1035,10,682,10,219,219,27,4294732,235,235,...,0,1,0,0,0,0,0,1,0,0


In [424]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14]),
 array([  632,  1594,   819, 35000,  2108, 17171, 27167, 35000,    35,
          266, 27031,   354, 15014]))

In [425]:
new_Benign.shape

(35000, 45)

In [426]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
126302,1744,20,26454,32,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
163033,126,2,190,2,0,0,0,4294952,15,0,...,0,0,1,0,0,0,0,0,1,0
191966,80,2,0,0,2,2,0,4294733,234,0,...,0,0,0,0,0,1,0,0,0,0
263943,655,9,3428,7,27,27,27,0,0,0,...,0,0,0,1,0,0,0,0,0,0
126519,581,5,420,6,30,26,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [427]:
df.shape

(197191, 45)

In [428]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14]),
 array([35000,   632,  1594,   819, 35000,  2108, 17171, 27167, 35000,
           35,   266, 27031,   354, 15014]))

In [429]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [430]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [431]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [432]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,1744,20,26454,32,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,126,2,190,2,0,0,0,4294952,15,0,...,0,0,1,0,0,0,0,0,1,0
2,80,2,0,0,2,2,0,4294733,234,0,...,0,0,0,0,0,1,0,0,0,0
3,655,9,3428,7,27,27,27,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,581,5,420,6,30,26,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [433]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [434]:
df_non = df
df_non['Attack'].values[:] = 0

In [435]:
df_non.shape

(490000, 45)

In [436]:
df = pd.concat([df_FTP,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,780,13,520,13,22,2,20,4294763,204,204,...,0,0,0,0,0,0,0,0,1,11
1,780,13,520,13,22,2,20,4294779,188,188,...,0,0,0,0,0,0,0,0,1,11
2,840,14,560,14,22,2,20,4294732,235,235,...,0,0,0,0,0,0,0,0,1,11
3,840,14,560,14,22,2,20,4294748,219,219,...,0,0,0,0,0,0,0,0,1,11
4,840,14,560,14,22,2,20,4294717,250,250,...,0,0,0,0,0,0,0,0,1,11


In [437]:
df.shape

(990000, 45)

In [438]:
np.unique(df['Attack'],return_counts = True)

(array([ 0, 11]), array([490000, 500000]))

### XGBoost Model of FTP-Brute Force vs. Others

In [439]:
df.replace({'Attack':11},{'Attack':1},inplace = True)

In [440]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,780,13,520,13,22,2,20,4294763,204,204,...,0,0,0,0,0,0,0,0,1,1
1,780,13,520,13,22,2,20,4294779,188,188,...,0,0,0,0,0,0,0,0,1,1
2,840,14,560,14,22,2,20,4294732,235,235,...,0,0,0,0,0,0,0,0,1,1
3,840,14,560,14,22,2,20,4294748,219,219,...,0,0,0,0,0,0,0,0,1,1
4,840,14,560,14,22,2,20,4294717,250,250,...,0,0,0,0,0,0,0,0,1,1


In [441]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [682]:
model_FTP = xgb.XGBClassifier()
# Train the model
model_FTP.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [683]:
y_pred = model_FTP.predict(X_test)

In [684]:
results = classification_report(y_test,y_pred,output_dict = True)

In [685]:
df = pd.DataFrame(results).transpose()

In [686]:
df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


### Random Forest of FTP-Brute Force vs. Others

In [442]:
model_FTP = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_FTP.fit(X_train,y_train)

/tmp/ipykernel_729329/322440184.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_FTP.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [443]:
y_pred = model_FTP.predict(X_test)

In [444]:
results = classification_report(y_test,y_pred,output_dict = True)

In [445]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


In [447]:
model_pkl_file = "FTP-Brute Force RF Model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_FTP, file)

### LightGBM Model of FTP-BruteForce

In [691]:
model_FTP = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_FTP.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_FTP.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4967
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


## DDOS Attack - HOIC Model

In [448]:
query_HOIC = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 4"

In [449]:
cursor.execute(query_HOIC)

In [450]:
data = cursor.fetchall()

In [451]:
df_HOIC = pd.DataFrame(data = data, columns = colnames)

In [452]:
df_HOIC.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,522,5,1147,5,219,219,27,4294963,3,3,...,0,0,0,0,1,0,0,0,0,0
1,526,5,1147,5,219,219,27,4294925,41,41,...,0,0,0,0,1,0,0,0,0,0
2,521,5,1147,5,219,219,27,4294768,198,198,...,0,0,0,0,1,0,0,0,0,0
3,499,5,1147,5,219,219,27,4294936,30,30,...,0,0,0,0,1,0,0,0,0,0
4,479,5,1147,5,219,219,27,4294956,10,10,...,0,0,0,0,1,0,0,0,0,0


In [453]:
df_HOIC.shape

(98985, 45)

### Fetching Other Types from Database

In [454]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 500000"

In [455]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [456]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,3019,17,5150,19,222,222,90,4294733,234,234,...,0,0,0,1,0,0,0,0,0,1
1,2622,10,1979,11,222,222,26,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,40,1,0,0,2,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,40,1,0,0,2,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
4,1657,15,4297,13,30,30,26,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [457]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [458]:
df = pd.concat([df_HOIC,df_Benign])
df.reset_index(inplace = True,drop = True)

In [459]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,522,5,1147,5,219,219,27,4294963,3,3,...,0,0,0,0,1,0,0,0,0,0
1,526,5,1147,5,219,219,27,4294925,41,41,...,0,0,0,0,1,0,0,0,0,0
2,521,5,1147,5,219,219,27,4294768,198,198,...,0,0,0,0,1,0,0,0,0,0
3,499,5,1147,5,219,219,27,4294936,30,30,...,0,0,0,0,1,0,0,0,0,0
4,479,5,1147,5,219,219,27,4294956,10,10,...,0,0,0,0,1,0,0,0,0,0


In [460]:
df.shape

(598985, 45)

In [461]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [462]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,522,5,1147,5,219,219,27,4294963,3,3,...,0,0,0,0,1,0,0,0,0,0
1,526,5,1147,5,219,219,27,4294925,41,41,...,0,0,0,0,1,0,0,0,0,0
2,521,5,1147,5,219,219,27,4294768,198,198,...,0,0,0,0,1,0,0,0,0,0
3,499,5,1147,5,219,219,27,4294936,30,30,...,0,0,0,0,1,0,0,0,0,0
4,479,5,1147,5,219,219,27,4294956,10,10,...,0,0,0,0,1,0,0,0,0,0


In [463]:
Y.head()

,Attack
0,4
1,4
2,4
3,4
4,4


In [464]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [465]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [466]:
np.unique(y_resampled,return_counts = True)

(array([0, 4]), array([500000, 500000]))

In [467]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,522,5,1147,5,219,219,27,4294963,3,3,...,0,0,0,1,0,0,0,0,0,4
1,526,5,1147,5,219,219,27,4294925,41,41,...,0,0,0,1,0,0,0,0,0,4
2,521,5,1147,5,219,219,27,4294768,198,198,...,0,0,0,1,0,0,0,0,0,4
3,499,5,1147,5,219,219,27,4294936,30,30,...,0,0,0,1,0,0,0,0,0,4
4,479,5,1147,5,219,219,27,4294956,10,10,...,0,0,0,1,0,0,0,0,0,4


### Undersampling Benign Records Using Numpy Sample

In [468]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [469]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
225240,64,1,258,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
261971,1768,35,61262,61,31,31,30,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0
290904,2210,22,6431,23,31,31,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
362881,1809,36,49109,58,31,31,30,4294952,15,15,...,0,0,0,0,0,0,0,0,1,0
225457,73,1,138,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [470]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [471]:
df_HOIC = df_new[df_new['Attack']==4]
df_HOIC.shape

(500000, 45)

In [472]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [473]:
labels = [1,2,3,5,6,7,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [474]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1076,10,693,10,219,219,27,4294717,250,250,...,0,1,0,0,0,0,0,1,0,0
1,1077,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
2,172,4,3261,78,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,229,5,821,17,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1077,10,682,10,219,219,27,4294717,250,250,...,0,1,0,0,0,0,0,1,0,0


In [475]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([  632,  1594,   819,  2108, 17171, 27167, 35000,    35,   266,
           47, 27031,   354, 15014]))

In [476]:
new_Benign.shape

(35000, 45)

In [477]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
225240,64,1,258,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
261971,1768,35,61262,61,31,31,30,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0
290904,2210,22,6431,23,31,31,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
362881,1809,36,49109,58,31,31,30,4294952,15,15,...,0,0,0,0,0,0,0,0,1,0
225457,73,1,138,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [478]:
df.shape

(162238, 45)

In [479]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000,   632,  1594,   819,  2108, 17171, 27167, 35000,    35,
          266,    47, 27031,   354, 15014]))

In [480]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [481]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [482]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [483]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,64,1,258,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,1768,35,61262,61,31,31,30,4294951,16,16,...,0,0,0,0,0,0,0,0,1,0
2,2210,22,6431,23,31,31,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1809,36,49109,58,31,31,30,4294952,15,15,...,0,0,0,0,0,0,0,0,1,0
4,73,1,138,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [484]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [485]:
df_non = df
df_non['Attack'].values[:] = 0

In [486]:
df_non.shape

(490000, 45)

In [487]:
df = pd.concat([df_HOIC,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,522,5,1147,5,219,219,27,4294963,3,3,...,0,0,0,1,0,0,0,0,0,4
1,526,5,1147,5,219,219,27,4294925,41,41,...,0,0,0,1,0,0,0,0,0,4
2,521,5,1147,5,219,219,27,4294768,198,198,...,0,0,0,1,0,0,0,0,0,4
3,499,5,1147,5,219,219,27,4294936,30,30,...,0,0,0,1,0,0,0,0,0,4
4,479,5,1147,5,219,219,27,4294956,10,10,...,0,0,0,1,0,0,0,0,0,4


In [488]:
df.shape

(990000, 45)

In [489]:
np.unique(df['Attack'],return_counts = True)

(array([0, 4]), array([490000, 500000]))

### XGBoost Model of DoS Attacks-HOIC vs. Others

In [490]:
df.replace({'Attack':4},{'Attack':1},inplace = True)

In [491]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,522,5,1147,5,219,219,27,4294963,3,3,...,0,0,0,1,0,0,0,0,0,1
1,526,5,1147,5,219,219,27,4294925,41,41,...,0,0,0,1,0,0,0,0,0,1
2,521,5,1147,5,219,219,27,4294768,198,198,...,0,0,0,1,0,0,0,0,0,1
3,499,5,1147,5,219,219,27,4294936,30,30,...,0,0,0,1,0,0,0,0,0,1
4,479,5,1147,5,219,219,27,4294956,10,10,...,0,0,0,1,0,0,0,0,0,1


In [492]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [737]:
model_HOIC = xgb.XGBClassifier()
# Train the model
model_HOIC.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [738]:
y_pred = model_HOIC.predict(X_test)

In [739]:
results = classification_report(y_test,y_pred,output_dict = True)

In [740]:
df = pd.DataFrame(results).transpose()

In [741]:
df

,precision,recall,f1-score,support
0,0.977351,0.968568,0.972940,97927.000000
1,0.969511,0.978036,0.973755,100073.000000
accuracy,0.973354,0.973354,0.973354,0.973354
macro avg,0.973431,0.973302,0.973347,198000.000000
weighted avg,0.973388,0.973354,0.973352,198000.000000


### Random Forest of DoS Attacks-HOIC vs. Others

In [493]:
model_HOIC = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_HOIC.fit(X_train,y_train)

/tmp/ipykernel_729329/1693603200.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_HOIC.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [494]:
y_pred = model_HOIC.predict(X_test)

In [495]:
results = classification_report(y_test,y_pred,output_dict = True)

In [496]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,0.984703,0.976830,0.980751,97927.000000
1,0.977503,0.985151,0.981312,100073.000000
accuracy,0.981035,0.981035,0.981035,0.981035
macro avg,0.981103,0.980990,0.981031,198000.000000
weighted avg,0.981064,0.981035,0.981034,198000.000000


In [497]:
model_pkl_file = "DDoS Attacks-HOIC RF Model.pkl"  

with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_HOIC, file)

### LightGBM Model of DDoS Attacks-HOIC

In [746]:
model_HOIC = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_HOIC.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_HOIC.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015138 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4973
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834


,precision,recall,f1-score,support
0,0.971934,0.958694,0.965268,97927.000000
1,0.960111,0.972910,0.966468,100073.000000
accuracy,0.965879,0.965879,0.965879,0.965879
macro avg,0.966023,0.965802,0.965868,198000.000000
weighted avg,0.965958,0.965879,0.965875,198000.000000


## DoS-attacks-Slowloris Model

### Fetching SQL Data from Database

In [498]:
query_model_DoS_attacks_Slowloris = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE ATTACK = 10"
cursor.execute(query_model_DoS_attacks_Slowloris)
data = cursor.fetchall()
df_model_DoS_attacks_Slowloris = pd.DataFrame(data = data, columns = colnames)
df_model_DoS_attacks_Slowloris.Attack.value_counts()
df_model_DoS_attacks_Slowloris.shape

(266, 45)

### Fetching Other Types from Database

In [499]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE ATTACK = 0 ORDER BY RAND() LIMIT 500000"
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)
df_Benign.head()
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [500]:
df = pd.concat([df_model_DoS_attacks_Slowloris,df_Benign])
df.reset_index(inplace = True,drop = True)
df.head()
df.shape
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X.head()
Y.head()
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)
X_resampled,y_resampled = smote.fit_resample(X,Y)
np.unique(y_resampled,return_counts = True)
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,3326,15,160,3,30,26,22,0,0,0,...,0,0,0,1,0,0,0,0,0,10
1,180,3,0,0,2,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,10
2,3266,14,160,3,30,26,22,4294952,15,15,...,0,0,0,1,0,0,0,0,0,10
3,3176,15,160,3,30,26,22,0,0,0,...,0,0,0,1,0,0,0,0,0,10
4,3192,17,740,4,31,31,27,4294952,15,15,...,0,0,0,1,0,0,0,0,0,10


### Undersampling Benign Records Using Numpy Sample

In [501]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)
new_Benign.head()
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks
Set them all to have 35000 instances

In [502]:
df_model_DoS_attacks_Slowloris = df_new[df_new['Attack']==10]
df_model_DoS_attacks_Slowloris.shape

q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]
labels = [1,2,3,4,5,6,7,8,9,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

df = pd.DataFrame(data = data, columns = colnames)
df.head()
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14]),
 array([  632,  1594,   819, 35000,  2108, 17171, 27167, 35000,    35,
           47, 27031,   354, 15014]))

In [503]:
new_Benign.shape
df = pd.concat([new_Benign,df])
df.head()
df.shape
np.unique(df['Attack'],return_counts = True)
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)
X_resampled,y_resampled = smote.fit_resample(X,Y)
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()
np.unique(df['Attack'],return_counts = True)
df_non = df
df_non['Attack'].values[:] = 0
df_non.shape
df = pd.concat([df_model_DoS_attacks_Slowloris,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,3326,15,160,3,30,26,22,0,0,0,...,0,0,0,1,0,0,0,0,0,10
1,180,3,0,0,2,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,10
2,3266,14,160,3,30,26,22,4294952,15,15,...,0,0,0,1,0,0,0,0,0,10
3,3176,15,160,3,30,26,22,0,0,0,...,0,0,0,1,0,0,0,0,0,10
4,3192,17,740,4,31,31,27,4294952,15,15,...,0,0,0,1,0,0,0,0,0,10


In [504]:
df.shape
np.unique(df['Attack'],return_counts = True)

(array([ 0, 10]), array([490000, 500000]))

### XGBoost Model of model_DoS_attacks_Slowloris vs. Others

In [483]:
df.replace({'Attack':10},{'Attack':1},inplace = True)
df.head()
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)
model_model_DoS_attacks_Slowloris = xgb.XGBClassifier()
# Train the model
model_model_DoS_attacks_Slowloris.fit(X_train,y_train)
y_pred = model_model_DoS_attacks_Slowloris.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


### Random Forest Model of model_DoS_attacks_Slowloris vs. Others


In [505]:
model_model_DoS_attacks_Slowloris = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_model_DoS_attacks_Slowloris.fit(X_train,y_train)
y_pred = model_model_DoS_attacks_Slowloris.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

/tmp/ipykernel_729329/4071675711.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_model_DoS_attacks_Slowloris.fit(X_train,y_train)


,precision,recall,f1-score,support
0,0.984703,0.976830,0.980751,97927.000000
1,0.977503,0.985151,0.981312,100073.000000
accuracy,0.981035,0.981035,0.981035,0.981035
macro avg,0.981103,0.980990,0.981031,198000.000000
weighted avg,0.981064,0.981035,0.981034,198000.000000


In [507]:
model_pkl_file = "DoS Attacks-Slowloris RF Model.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_model_DoS_attacks_Slowloris, file)

### LightGBM of DoS-attacks-Slowloris Model

In [486]:
model_Slowloris = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_Slowloris.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_Slowloris.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018228 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5045
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [487]:
df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


## DoS Attacks-Hulk Model

### Fetching DoS attacks-Hulk Data from Database

In [6]:
query_DoS_attacks_Hulk = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE ATTACK = 8"
cursor.execute(query_DoS_attacks_Hulk)
data = cursor.fetchall()
df_DoS_attacks_Hulk = pd.DataFrame(data = data, columns = colnames)
df_DoS_attacks_Hulk.Attack.value_counts()
df_DoS_attacks_Hulk.shape

(324182, 45)

### Fetching Other Types from Database

In [7]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE ATTACK = 0 ORDER BY RAND() LIMIT 500000"

In [8]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [9]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,707,6,528,5,30,26,30,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1460,8,2070,11,222,222,90,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1875,13,1991,10,223,219,30,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,1514,21,8582,23,223,223,30,4294952,15,15,...,0,0,0,0,0,0,0,0,0,1
4,68,1,150,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [10]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [11]:
df = pd.concat([df_DoS_attacks_Hulk,df_Benign])
df.reset_index(inplace = True,drop = True)

In [12]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,3192,27,6015,25,27,27,27,4294655,312,312,...,0,0,0,0,1,0,0,0,0,0
1,3569,29,5963,24,27,27,27,4294686,281,281,...,0,0,0,0,1,0,0,0,0,0
2,2869,22,4812,20,27,27,27,4294764,203,203,...,0,0,0,0,1,0,0,0,0,0
3,3275,25,6015,25,27,27,27,4294717,250,250,...,0,0,0,0,1,0,0,0,0,0
4,3516,27,6015,25,27,27,27,4294686,281,281,...,0,0,0,0,1,0,0,0,0,0


In [13]:
df.shape

(824182, 45)

In [14]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [15]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,3192,27,6015,25,27,27,27,4294655,312,312,...,0,0,0,0,1,0,0,0,0,0
1,3569,29,5963,24,27,27,27,4294686,281,281,...,0,0,0,0,1,0,0,0,0,0
2,2869,22,4812,20,27,27,27,4294764,203,203,...,0,0,0,0,1,0,0,0,0,0
3,3275,25,6015,25,27,27,27,4294717,250,250,...,0,0,0,0,1,0,0,0,0,0
4,3516,27,6015,25,27,27,27,4294686,281,281,...,0,0,0,0,1,0,0,0,0,0


In [16]:
Y.head()

,Attack
0,8
1,8
2,8
3,8
4,8


In [17]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [18]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [19]:
np.unique(y_resampled,return_counts = True)

(array([0, 8]), array([500000, 500000]))

In [20]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,3192,27,6015,25,27,27,27,4294655,312,312,...,0,0,0,1,0,0,0,0,0,8
1,3569,29,5963,24,27,27,27,4294686,281,281,...,0,0,0,1,0,0,0,0,0,8
2,2869,22,4812,20,27,27,27,4294764,203,203,...,0,0,0,1,0,0,0,0,0,8
3,3275,25,6015,25,27,27,27,4294717,250,250,...,0,0,0,1,0,0,0,0,0,8
4,3516,27,6015,25,27,27,27,4294686,281,281,...,0,0,0,1,0,0,0,0,0,8


### Undersampling Benign Records Using Numpy Sample

In [21]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [22]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
450437,2343,22,7328,24,223,223,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
487168,40,1,0,0,2,2,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
516101,900,8,8816,14,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
588078,1224,18,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
450654,1478,7,1536,9,219,219,27,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [23]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks
Set them all to have 35000 instances

In [24]:
df_DoS_attacks_Hulk = df_new[df_new['Attack']==8]
df_DoS_attacks_Hulk.shape

(500000, 45)

In [25]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [26]:
labels = [1,2,3,4,5,6,7,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [27]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,172,4,2981,71,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1076,10,682,10,219,219,27,4294733,234,234,...,0,1,0,0,0,0,0,1,0,0
2,172,4,1621,37,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,172,4,4021,97,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1035,10,682,10,219,219,27,4294717,250,250,...,0,1,0,0,0,0,0,1,0,0


In [28]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14]),
 array([  632,  1594,   819, 35000,  2108, 17171, 27167,    35,   266,
           47, 27031,   354, 15014]))

In [29]:
new_Benign.shape

(35000, 45)

In [30]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
450437,2343,22,7328,24,223,223,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
487168,40,1,0,0,2,2,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
516101,900,8,8816,14,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
588078,1224,18,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
450654,1478,7,1536,9,219,219,27,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [31]:
df.shape

(162238, 45)

In [32]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14]),
 array([35000,   632,  1594,   819, 35000,  2108, 17171, 27167,    35,
          266,    47, 27031,   354, 15014]))

In [33]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [34]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [35]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [36]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,2343,22,7328,24,223,223,30,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,40,1,0,0,2,2,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,900,8,8816,14,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1224,18,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,1478,7,1536,9,219,219,27,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [37]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [38]:
df_non = df
df_non['Attack'].values[:] = 0

In [39]:
df_non.shape

(490000, 45)

In [40]:
df = pd.concat([df_DoS_attacks_Hulk,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,3192,27,6015,25,27,27,27,4294655,312,312,...,0,0,0,1,0,0,0,0,0,8
1,3569,29,5963,24,27,27,27,4294686,281,281,...,0,0,0,1,0,0,0,0,0,8
2,2869,22,4812,20,27,27,27,4294764,203,203,...,0,0,0,1,0,0,0,0,0,8
3,3275,25,6015,25,27,27,27,4294717,250,250,...,0,0,0,1,0,0,0,0,0,8
4,3516,27,6015,25,27,27,27,4294686,281,281,...,0,0,0,1,0,0,0,0,0,8


In [41]:
df.shape

(990000, 45)

In [42]:
np.unique(df['Attack'],return_counts = True)

(array([0, 8]), array([490000, 500000]))

### XGBoost Model of DoS_attacks_Hulk vs. Others

In [43]:
df.replace({'Attack':8},{'Attack':1},inplace = True)

In [44]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,3192,27,6015,25,27,27,27,4294655,312,312,...,0,0,0,1,0,0,0,0,0,1
1,3569,29,5963,24,27,27,27,4294686,281,281,...,0,0,0,1,0,0,0,0,0,1
2,2869,22,4812,20,27,27,27,4294764,203,203,...,0,0,0,1,0,0,0,0,0,1
3,3275,25,6015,25,27,27,27,4294717,250,250,...,0,0,0,1,0,0,0,0,0,1
4,3516,27,6015,25,27,27,27,4294686,281,281,...,0,0,0,1,0,0,0,0,0,1


In [45]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [47]:
model_DoS_attacks_Hulk = xgb.XGBClassifier()
# Train the model
model_DoS_attacks_Hulk.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [48]:
y_pred = model_DoS_attacks_Hulk.predict(X_test)

In [49]:
results = classification_report(y_test,y_pred,output_dict = True)

In [50]:
df = pd.DataFrame(results).transpose()

In [51]:
df

,precision,recall,f1-score,support
0,0.99998,1.00000,0.99999,97927.00000
1,1.00000,0.99998,0.99999,100073.00000
accuracy,0.99999,0.99999,0.99999,0.99999
macro avg,0.99999,0.99999,0.99999,198000.00000
weighted avg,0.99999,0.99999,0.99999,198000.00000


### Random Forest of DoS Attacks-Hulk vs. Others

In [46]:
model_dos_hulk = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_dos_hulk.fit(X_train,y_train)
y_pred = model_dos_hulk.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

/tmp/ipykernel_816085/55468040.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_dos_hulk.fit(X_train,y_train)


,precision,recall,f1-score,support
0,0.99998,1.00000,0.99999,97927.00000
1,1.00000,0.99998,0.99999,100073.00000
accuracy,0.99999,0.99999,0.99999,0.99999
macro avg,0.99999,0.99999,0.99999,198000.00000
weighted avg,0.99999,0.99999,0.99999,198000.00000


In [239]:
model_pkl_file = "DoS Attacks-Hulk RF Model.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_dos_hulk, file)

### LightGBM Model of DoS-attacks-Hulk

In [53]:
model_dos_hulk = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_dos_hulk.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_dos_hulk.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.015714 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4933
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834


,precision,recall,f1-score,support
0,0.99998,1.00000,0.99999,97927.00000
1,1.00000,0.99998,0.99999,100073.00000
accuracy,0.99999,0.99999,0.99999,0.99999
macro avg,0.99999,0.99999,0.99999,198000.00000
weighted avg,0.99999,0.99999,0.99999,198000.00000


## DoS-Attacks SlowHTTPTest

In [48]:
query_DoS_attacks_SlowHTTPTest = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE ATTACK = 9"

In [49]:
cursor.execute(query_DoS_attacks_SlowHTTPTest)

In [50]:
data = cursor.fetchall()

In [51]:
df_DoS_attacks_SlowHTTPTest = pd.DataFrame(data = data, columns = colnames)

In [52]:
df_DoS_attacks_SlowHTTPTest.Attack.value_counts()

9    35
Name: Attack, dtype: int64

In [53]:
df_DoS_attacks_SlowHTTPTest.shape

(35, 45)

### Fetching Other Types from Database

In [54]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE ATTACK = 0 ORDER BY RAND() LIMIT 500000"

In [55]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [56]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,2515,20,1571,16,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,44,1,0,0,2,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
2,1418,10,8268,12,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2560,28,111601,98,219,219,91,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,4223,25,7386,27,223,223,30,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [57]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [58]:
df = pd.concat([df_DoS_attacks_SlowHTTPTest,df_Benign])
df.reset_index(inplace = True,drop = True)

In [59]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,420,7,280,7,22,2,20,4294873,94,94,...,0,0,0,0,0,0,0,0,0,1
1,480,8,320,8,22,2,20,4294826,141,141,...,0,0,0,0,0,0,0,0,0,1
2,420,7,280,7,22,2,20,4294858,109,109,...,0,0,0,0,0,0,0,0,0,1
3,420,7,280,7,22,2,20,4294842,125,125,...,0,0,0,0,0,0,0,0,0,1
4,480,8,320,8,22,2,20,4294811,156,156,...,0,0,0,0,0,0,0,0,0,1


In [60]:
df.shape

(500035, 45)

In [61]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [62]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,420,7,280,7,22,2,20,4294873,94,94,...,0,0,0,0,0,0,0,0,0,1
1,480,8,320,8,22,2,20,4294826,141,141,...,0,0,0,0,0,0,0,0,0,1
2,420,7,280,7,22,2,20,4294858,109,109,...,0,0,0,0,0,0,0,0,0,1
3,420,7,280,7,22,2,20,4294842,125,125,...,0,0,0,0,0,0,0,0,0,1
4,480,8,320,8,22,2,20,4294811,156,156,...,0,0,0,0,0,0,0,0,0,1


In [63]:
Y.head()

,Attack
0,9
1,9
2,9
3,9
4,9


In [64]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [65]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [66]:
np.unique(y_resampled,return_counts = True)

(array([0, 9]), array([500000, 500000]))

In [67]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,420,7,280,7,22,2,20,4294873,94,94,...,0,0,0,0,0,0,0,0,1,9
1,480,8,320,8,22,2,20,4294826,141,141,...,0,0,0,0,0,0,0,0,1,9
2,420,7,280,7,22,2,20,4294858,109,109,...,0,0,0,0,0,0,0,0,1,9
3,420,7,280,7,22,2,20,4294842,125,125,...,0,0,0,0,0,0,0,0,1,9
4,480,8,320,8,22,2,20,4294811,156,156,...,0,0,0,0,0,0,0,0,1,9


### Undersampling Benign Records Using Numpy Sample

In [68]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [69]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
126290,1861,22,38691,39,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
163021,3408,21,5181,19,222,222,218,0,0,0,...,0,0,1,0,0,0,0,0,1,0
191954,2654,14,4566,14,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0
263931,3261,24,5911,22,31,31,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
126507,652,7,870,7,31,27,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [70]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [71]:
df_DoS_attacks_SlowHTTPTest = df_new[df_new['Attack']==9]
df_DoS_attacks_SlowHTTPTest.shape

(500000, 45)

In [72]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [73]:
labels = [1,2,3,4,5,6,7,8,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [74]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1672,5,981,21,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1548,5,1981,46,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,577,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1076,10,682,10,219,219,27,4294733,234,234,...,0,1,0,0,0,0,0,1,0,0
4,1076,10,682,10,219,219,27,4294716,251,251,...,0,1,0,0,0,0,0,1,0,0


In [75]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14]),
 array([  632,  1594,   819, 35000,  2108, 17171, 27167, 35000,   266,
           47, 27031,   354, 15014]))

In [76]:
new_Benign.shape

(35000, 45)

In [77]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
126290,1861,22,38691,39,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
163021,3408,21,5181,19,222,222,218,0,0,0,...,0,0,1,0,0,0,0,0,1,0
191954,2654,14,4566,14,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0
263931,3261,24,5911,22,31,31,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
126507,652,7,870,7,31,27,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [78]:
df.shape

(197203, 45)

In [79]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14]),
 array([35000,   632,  1594,   819, 35000,  2108, 17171, 27167, 35000,
          266,    47, 27031,   354, 15014]))

In [80]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [81]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [82]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [83]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,1861,22,38691,39,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,3408,21,5181,19,222,222,218,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,2654,14,4566,14,222,222,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,3261,24,5911,22,31,31,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,652,7,870,7,31,27,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [84]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [85]:
df_non = df
df_non['Attack'].values[:] = 0

In [86]:
df_non.shape

(490000, 45)

In [87]:
df = pd.concat([df_DoS_attacks_SlowHTTPTest,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,420,7,280,7,22,2,20,4294873,94,94,...,0,0,0,0,0,0,0,0,1,9
1,480,8,320,8,22,2,20,4294826,141,141,...,0,0,0,0,0,0,0,0,1,9
2,420,7,280,7,22,2,20,4294858,109,109,...,0,0,0,0,0,0,0,0,1,9
3,420,7,280,7,22,2,20,4294842,125,125,...,0,0,0,0,0,0,0,0,1,9
4,480,8,320,8,22,2,20,4294811,156,156,...,0,0,0,0,0,0,0,0,1,9


In [88]:
df.shape

(990000, 45)

In [89]:
np.unique(df['Attack'],return_counts = True)

(array([0, 9]), array([490000, 500000]))

In [90]:
df.replace({'Attack':9},{'Attack':1},inplace = True)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,420,7,280,7,22,2,20,4294873,94,94,...,0,0,0,0,0,0,0,0,1,1
1,480,8,320,8,22,2,20,4294826,141,141,...,0,0,0,0,0,0,0,0,1,1
2,420,7,280,7,22,2,20,4294858,109,109,...,0,0,0,0,0,0,0,0,1,1
3,420,7,280,7,22,2,20,4294842,125,125,...,0,0,0,0,0,0,0,0,1,1
4,480,8,320,8,22,2,20,4294811,156,156,...,0,0,0,0,0,0,0,0,1,1


### XGBoost Model of DoS_attacks_SlowHTTPTest vs. Others

In [91]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [107]:
model_DoS_attacks_SlowHTTPTest = xgb.XGBClassifier()
# Train the model
model_DoS_attacks_SlowHTTPTest.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [108]:
y_pred = model_DoS_attacks_SlowHTTPTest.predict(X_test)

In [109]:
results = classification_report(y_test,y_pred,output_dict = True)

In [110]:
df = pd.DataFrame(results).transpose()

In [111]:
df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


### Random Forest Model of DoS_attacks_SlowHTTPTest vs. Others

In [92]:
model_DoS_attacks_SlowHTTPTest = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_DoS_attacks_SlowHTTPTest.fit(X_train,y_train)
y_pred = model_DoS_attacks_SlowHTTPTest.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

/tmp/ipykernel_816085/289621487.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_DoS_attacks_SlowHTTPTest.fit(X_train,y_train)


,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


In [93]:
model_pkl_file = "DoS Attacks-SlowHTTPTest RF Model.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_DoS_attacks_SlowHTTPTest, file)

### DoS-Atacks-SlowHTTPTest LightGBM Model

In [113]:
model_DoS_attacks_SlowHTTPTest = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_DoS_attacks_SlowHTTPTest.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_DoS_attacks_SlowHTTPTest.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018935 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4966
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


## Bot Model

### Fetching Bot Data from Database

In [94]:
query_bot = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 1"

In [95]:
cursor.execute(query_bot)

In [96]:
data = cursor.fetchall()

In [97]:
df_bot = pd.DataFrame(data = data, columns = colnames)

In [98]:
df_bot.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,538,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1076,10,682,10,219,219,27,4294732,235,235,...,0,1,0,0,0,0,0,1,0,0
2,1076,10,682,10,219,219,27,4294717,250,250,...,0,1,0,0,0,0,0,1,0,0
3,497,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1076,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0


In [99]:
df_bot.shape 

(632, 45)

### Fetching Other Types from Database

In [100]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 500000"

In [101]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [102]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1480,8,1873,7,222,222,90,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1759,22,5236,21,31,31,30,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2840,27,4068,26,27,27,27,4294952,15,15,...,0,0,0,0,1,0,0,0,0,0
3,1868,19,6331,17,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,33308,102,21882,108,223,223,30,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [103]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [104]:
df = pd.concat([df_bot,df_Benign])
df.reset_index(inplace = True,drop = True)

In [105]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,538,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1076,10,682,10,219,219,27,4294732,235,235,...,0,1,0,0,0,0,0,1,0,0
2,1076,10,682,10,219,219,27,4294717,250,250,...,0,1,0,0,0,0,0,1,0,0
3,497,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1076,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0


In [106]:
df.shape

(500632, 45)

In [107]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [108]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,538,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1076,10,682,10,219,219,27,4294732,235,235,...,0,1,0,0,0,0,0,1,0,0
2,1076,10,682,10,219,219,27,4294717,250,250,...,0,1,0,0,0,0,0,1,0,0
3,497,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
4,1076,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0


In [109]:
Y.head()

,Attack
0,1
1,1
2,1
3,1
4,1


In [110]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [111]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [112]:
np.unique(y_resampled,return_counts = True)

(array([0, 1]), array([500000, 500000]))

In [113]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,538,5,341,5,219,219,27,0,0,0,...,1,0,0,0,0,0,1,0,0,1
1,1076,10,682,10,219,219,27,4294732,235,235,...,1,0,0,0,0,0,1,0,0,1
2,1076,10,682,10,219,219,27,4294717,250,250,...,1,0,0,0,0,0,1,0,0,1
3,497,5,341,5,219,219,27,0,0,0,...,1,0,0,0,0,0,1,0,0,1
4,1076,10,682,10,219,219,27,4294701,266,266,...,1,0,0,0,0,0,1,0,0,1


### Undersampling Benign Records Using Numpy Sample

In [114]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [115]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
126887,1828,20,4868,18,219,219,27,4294959,8,8,...,0,0,0,0,0,0,0,0,1,0
163618,5423,18,6977,14,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
192551,2068,27,26489,33,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
264528,2815,29,10763,31,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
127104,656,7,875,7,31,27,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [116]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [117]:
df_bot = df_new[df_new['Attack']==1]
df_bot.shape

(500000, 45)

In [118]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [119]:
labels = [2,3,4,5,6,7,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [120]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,552,5,1147,5,219,219,27,4294805,161,161,...,0,0,0,0,1,0,0,0,0,0
1,528,5,1147,5,219,219,27,4294957,10,10,...,0,0,0,0,1,0,0,0,0,0
2,478,5,1147,5,219,219,27,4294964,2,2,...,0,0,0,0,1,0,0,0,0,0
3,511,5,1147,5,219,219,27,4294963,3,3,...,0,0,0,0,1,0,0,0,0,0
4,536,5,1147,5,219,219,27,4294940,26,26,...,0,0,0,0,1,0,0,0,0,0


In [121]:
np.unique(df['Attack'],return_counts=True)

(array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([ 1594,   819, 35000,  2108, 17171, 27167, 35000,    35,   266,
           47, 27031,   354, 15014]))

In [122]:
new_Benign.shape

(35000, 45)

In [123]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
126887,1828,20,4868,18,219,219,27,4294959,8,8,...,0,0,0,0,0,0,0,0,1,0
163618,5423,18,6977,14,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
192551,2068,27,26489,33,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
264528,2815,29,10763,31,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
127104,656,7,875,7,31,27,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [124]:
df.shape

(196606, 45)

In [125]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000,  1594,   819, 35000,  2108, 17171, 27167, 35000,    35,
          266,    47, 27031,   354, 15014]))

In [126]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [127]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [128]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [129]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,1828,20,4868,18,219,219,27,4294959,8,8,...,0,0,0,0,0,0,0,0,1,0
1,5423,18,6977,14,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2068,27,26489,33,27,27,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2815,29,10763,31,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,656,7,875,7,31,27,30,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [130]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [131]:
df_non = df
df_non['Attack'].values[:] = 0

In [132]:
df_non.shape

(490000, 45)

In [133]:
df = pd.concat([df_bot,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,538,5,341,5,219,219,27,0,0,0,...,1,0,0,0,0,0,1,0,0,1
1,1076,10,682,10,219,219,27,4294732,235,235,...,1,0,0,0,0,0,1,0,0,1
2,1076,10,682,10,219,219,27,4294717,250,250,...,1,0,0,0,0,0,1,0,0,1
3,497,5,341,5,219,219,27,0,0,0,...,1,0,0,0,0,0,1,0,0,1
4,1076,10,682,10,219,219,27,4294701,266,266,...,1,0,0,0,0,0,1,0,0,1


In [134]:
df.shape

(990000, 45)

In [135]:
np.unique(df['Attack'],return_counts = True)

(array([0, 1]), array([490000, 500000]))

### XGBoost Model of Bot vs. Others

In [136]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [158]:
model_bot = xgb.XGBClassifier()
# Train the model
model_bot.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [159]:
y_pred = model_bot.predict(X_test)

In [160]:
results = classification_report(y_test,y_pred,output_dict = True)

In [161]:
df = pd.DataFrame(results).transpose()

In [162]:
df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


### Random Forest of Bot vs. Others

In [137]:
model_bot = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_bot.fit(X_train,y_train)

/tmp/ipykernel_816085/1535800852.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_bot.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [138]:
y_pred = model_bot.predict(X_test)

In [139]:
results = classification_report(y_test,y_pred,output_dict = True)

In [140]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


In [141]:
model_pkl_file = "Bot RF Model.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_bot, file)

### Bot LightGBM Model

In [167]:
model_bot = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_bot.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_bot.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017109 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5003
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


## DDoS Attacks - LOIC - HTTP Model

### Fetching LOIC-HTTP Data from Database

In [142]:
query_LOIC = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 6"

In [143]:
cursor.execute(query_LOIC)

In [144]:
data = cursor.fetchall()

In [145]:
df_LOIC = pd.DataFrame(data = data, columns = colnames)

In [146]:
df_LOIC.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,232,5,1136,4,223,222,27,4294827,140,0,...,0,0,0,0,1,0,0,0,0,0
1,407880,6798,0,0,0,0,0,4294686,281,0,...,0,0,0,0,0,0,0,0,0,1
2,232,5,1136,4,223,222,27,4294951,16,0,...,0,0,0,0,1,0,0,0,0,0
3,232,5,1136,4,223,222,27,4294905,62,0,...,0,0,0,0,1,0,0,0,0,0
4,232,5,1136,4,223,222,27,4294904,63,0,...,0,0,0,0,1,0,0,0,0,0


In [147]:
df_LOIC.shape 

(17171, 45)

### Fetching Other Types from Database

In [148]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = 0 ORDER BY RAND() LIMIT 500000"

In [149]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [150]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1488,11,1721,7,30,30,26,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,970,13,2581,13,27,27,27,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,1569,17,6694,18,27,27,27,4294951,16,16,...,0,0,0,0,0,0,0,0,0,1
3,1986,14,835,10,223,219,94,4294952,0,15,...,0,0,0,1,0,0,0,0,0,1
4,2590,8,2896,7,27,27,27,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [151]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [152]:
df = pd.concat([df_LOIC,df_Benign])
df.reset_index(inplace = True,drop = True)

In [153]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,232,5,1136,4,223,222,27,4294827,140,0,...,0,0,0,0,1,0,0,0,0,0
1,407880,6798,0,0,0,0,0,4294686,281,0,...,0,0,0,0,0,0,0,0,0,1
2,232,5,1136,4,223,222,27,4294951,16,0,...,0,0,0,0,1,0,0,0,0,0
3,232,5,1136,4,223,222,27,4294905,62,0,...,0,0,0,0,1,0,0,0,0,0
4,232,5,1136,4,223,222,27,4294904,63,0,...,0,0,0,0,1,0,0,0,0,0


In [154]:
df.shape

(517171, 45)

In [155]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [156]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,232,5,1136,4,223,222,27,4294827,140,0,...,0,0,0,0,1,0,0,0,0,0
1,407880,6798,0,0,0,0,0,4294686,281,0,...,0,0,0,0,0,0,0,0,0,1
2,232,5,1136,4,223,222,27,4294951,16,0,...,0,0,0,0,1,0,0,0,0,0
3,232,5,1136,4,223,222,27,4294905,62,0,...,0,0,0,0,1,0,0,0,0,0
4,232,5,1136,4,223,222,27,4294904,63,0,...,0,0,0,0,1,0,0,0,0,0


In [157]:
Y.head()

,Attack
0,6
1,6
2,6
3,6
4,6


In [158]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [159]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [160]:
np.unique(y_resampled,return_counts = True)

(array([0, 6]), array([500000, 500000]))

In [161]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,232,5,1136,4,223,222,27,4294827,140,0,...,0,0,0,1,0,0,0,0,0,6
1,407880,6798,0,0,0,0,0,4294686,281,0,...,0,0,0,0,0,0,0,0,1,6
2,232,5,1136,4,223,222,27,4294951,16,0,...,0,0,0,1,0,0,0,0,0,6
3,232,5,1136,4,223,222,27,4294905,62,0,...,0,0,0,1,0,0,0,0,0,6
4,232,5,1136,4,223,222,27,4294904,63,0,...,0,0,0,1,0,0,0,0,0,6


### Undersampling Benign Records Using Numpy Sample

In [162]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [163]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
143426,1260,6,1638,5,219,219,27,0,0,0,...,0,0,0,1,0,0,0,0,0,0
180157,3333,12,7525,11,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
209090,1141,10,6253,12,26,26,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0
281067,70,1,86,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
143643,2316,11,4247,10,223,223,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [164]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [165]:
df_LOIC = df_new[df_new['Attack']==6]
df_LOIC.shape

(500000, 45)

In [166]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [167]:
labels = [1,2,3,4,5,7,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [168]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,172,4,581,11,219,211,27,4294952,15,15,...,0,1,0,0,0,0,0,1,0,0
1,497,5,341,5,219,219,27,4294952,15,15,...,0,1,0,0,0,0,0,1,0,0
2,1168,5,4621,112,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
3,1783,10,682,10,219,219,27,4294733,234,234,...,0,1,0,0,0,0,0,1,0,0
4,497,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [169]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([  632,  1594,   819, 35000,  2108, 27167, 35000,    35,   266,
           47, 27031,   354, 15014]))

In [170]:
new_Benign.shape

(35000, 45)

In [171]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
143426,1260,6,1638,5,219,219,27,0,0,0,...,0,0,0,1,0,0,0,0,0,0
180157,3333,12,7525,11,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
209090,1141,10,6253,12,26,26,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0
281067,70,1,86,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
143643,2316,11,4247,10,223,223,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [172]:
df.shape

(180067, 45)

In [173]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000,   632,  1594,   819, 35000,  2108, 27167, 35000,    35,
          266,    47, 27031,   354, 15014]))

In [174]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [175]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [176]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [177]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,1260,6,1638,5,219,219,27,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,3333,12,7525,11,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1141,10,6253,12,26,26,26,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,70,1,86,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,2316,11,4247,10,223,223,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [178]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [179]:
df_non = df
df_non['Attack'].values[:] = 0

In [180]:
df_non.shape

(490000, 45)

In [181]:
df = pd.concat([df_LOIC,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,232,5,1136,4,223,222,27,4294827,140,0,...,0,0,0,1,0,0,0,0,0,6
1,407880,6798,0,0,0,0,0,4294686,281,0,...,0,0,0,0,0,0,0,0,1,6
2,232,5,1136,4,223,222,27,4294951,16,0,...,0,0,0,1,0,0,0,0,0,6
3,232,5,1136,4,223,222,27,4294905,62,0,...,0,0,0,1,0,0,0,0,0,6
4,232,5,1136,4,223,222,27,4294904,63,0,...,0,0,0,1,0,0,0,0,0,6


In [182]:
df.shape

(990000, 45)

In [183]:
np.unique(df['Attack'],return_counts = True)

(array([0, 6]), array([490000, 500000]))

### XGBoost Model of DDOS Attacks - LOIC-HTTP vs. Others

In [184]:
df.replace({'Attack':6},{'Attack':1},inplace = True)

In [185]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,232,5,1136,4,223,222,27,4294827,140,0,...,0,0,0,1,0,0,0,0,0,1
1,407880,6798,0,0,0,0,0,4294686,281,0,...,0,0,0,0,0,0,0,0,1,1
2,232,5,1136,4,223,222,27,4294951,16,0,...,0,0,0,1,0,0,0,0,0,1
3,232,5,1136,4,223,222,27,4294905,62,0,...,0,0,0,1,0,0,0,0,0,1
4,232,5,1136,4,223,222,27,4294904,63,0,...,0,0,0,1,0,0,0,0,0,1


In [186]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [213]:
model_LOIC = xgb.XGBClassifier()
# Train the model
model_LOIC.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [214]:
y_pred = model_LOIC.predict(X_test)

In [215]:
results = classification_report(y_test,y_pred,output_dict = True)

In [216]:
df = pd.DataFrame(results).transpose()

In [217]:
df

,precision,recall,f1-score,support
0,0.999918,0.999765,0.999842,97927.000000
1,0.999770,0.999920,0.999845,100073.000000
accuracy,0.999843,0.999843,0.999843,0.999843
macro avg,0.999844,0.999843,0.999843,198000.000000
weighted avg,0.999843,0.999843,0.999843,198000.000000


### Random Forest of DDOS Attacks-LOIC-HTTP  vs. Others

In [187]:
model_LOIC = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_LOIC.fit(X_train,y_train)

/tmp/ipykernel_816085/1135490281.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_LOIC.fit(X_train,y_train)


RandomForestClassifier(n_estimators=30, random_state=30)

In [188]:
y_pred = model_LOIC.predict(X_test)

In [189]:
results = classification_report(y_test,y_pred,output_dict = True)

In [190]:
df = pd.DataFrame(results).transpose()
df

,precision,recall,f1-score,support
0,0.999908,0.999755,0.999831,97927.000000
1,0.999760,0.999910,0.999835,100073.000000
accuracy,0.999833,0.999833,0.999833,0.999833
macro avg,0.999834,0.999832,0.999833,198000.000000
weighted avg,0.999833,0.999833,0.999833,198000.000000


In [191]:
model_pkl_file = "DDoS Attacks-LOIC-HTTP RF Model.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_LOIC, file)

### DDOS Attacks-LOIC-HTTP LightGBM Model

In [222]:
model_LOIC = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_LOIC.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_LOIC.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022629 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5075
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834


,precision,recall,f1-score,support
0,0.999765,0.999602,0.999683,97927.000000
1,0.999610,0.999770,0.999690,100073.000000
accuracy,0.999687,0.999687,0.999687,0.999687
macro avg,0.999688,0.999686,0.999687,198000.000000
weighted avg,0.999687,0.999687,0.999687,198000.000000


## DoS Attacks-GoldenEye Model

In [192]:
query_DoS_attack = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE ATTACK = 7"

In [193]:
cursor.execute(query_DoS_attack)

In [194]:
data = cursor.fetchall()

In [195]:
df_DoS_attack = pd.DataFrame(data = data, columns = colnames)

In [196]:
df_DoS_attack.Attack.value_counts()

7    27167
Name: Attack, dtype: int64

In [197]:
df_DoS_attack.shape

(27167, 45)

### Fetching Other Types from Database

In [198]:
query_Benign = "SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE ATTACK = 0 ORDER BY RAND() LIMIT 500000"

In [199]:
cursor.execute(query_Benign)
data = cursor.fetchall()
df_Benign = pd.DataFrame(data = data, columns = colnames)

In [200]:
df_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,40,1,0,0,2,2,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
1,1621,15,8198,19,222,218,94,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,68,1,130,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1
3,919,10,4193,9,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,23444,113,20718,106,30,30,26,4294952,15,15,...,0,0,0,0,0,0,0,0,0,1


In [201]:
df_Benign.shape

(500000, 45)

### Concatenate DataFrames For SMOTE

In [202]:
df = pd.concat([df_DoS_attack,df_Benign])
df.reset_index(inplace = True,drop = True)

In [203]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1388,12,2066,8,31,30,27,4294077,890,875,...,0,0,0,0,1,0,0,0,0,0
1,1574,10,1962,6,30,30,26,4294685,282,282,...,0,0,0,0,1,0,0,0,0,0
2,4924,19,2014,7,31,30,27,4294764,203,203,...,0,0,0,0,1,0,0,0,0,0
3,1345,10,2272,6,30,30,26,4294701,266,266,...,0,0,0,0,1,0,0,0,0,0
4,1100,10,1962,6,30,30,26,4294248,719,719,...,0,0,0,0,1,0,0,0,0,0


In [204]:
df.shape

(527167, 45)

In [205]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [206]:
X.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1388,12,2066,8,31,30,27,4294077,890,875,...,0,0,0,0,1,0,0,0,0,0
1,1574,10,1962,6,30,30,26,4294685,282,282,...,0,0,0,0,1,0,0,0,0,0
2,4924,19,2014,7,31,30,27,4294764,203,203,...,0,0,0,0,1,0,0,0,0,0
3,1345,10,2272,6,30,30,26,4294701,266,266,...,0,0,0,0,1,0,0,0,0,0
4,1100,10,1962,6,30,30,26,4294248,719,719,...,0,0,0,0,1,0,0,0,0,0


In [207]:
Y.head()

,Attack
0,7
1,7
2,7
3,7
4,7


In [208]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [209]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [210]:
np.unique(y_resampled,return_counts = True)

(array([0, 7]), array([500000, 500000]))

In [211]:
df_new = pd.concat([X_resampled,y_resampled],axis = 1)
df_new.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,1388,12,2066,8,31,30,27,4294077,890,875,...,0,0,0,1,0,0,0,0,0,7
1,1574,10,1962,6,30,30,26,4294685,282,282,...,0,0,0,1,0,0,0,0,0,7
2,4924,19,2014,7,31,30,27,4294764,203,203,...,0,0,0,1,0,0,0,0,0,7
3,1345,10,2272,6,30,30,26,4294701,266,266,...,0,0,0,1,0,0,0,0,0,7
4,1100,10,1962,6,30,30,26,4294248,719,719,...,0,0,0,1,0,0,0,0,0,7


### Undersampling Benign Records Using Numpy Sample

In [212]:
new_Benign = df_new[df_new['Attack']==0].sample(35000,random_state = 30)

In [213]:
new_Benign.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
153422,3131,24,7780,25,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
190153,2955,23,2086,24,27,27,27,4294952,15,15,...,0,0,0,0,0,0,0,0,1,0
219086,2102,19,8385,19,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
291063,1384,8,1801,9,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0
153639,65,1,123,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [214]:
new_Benign.shape

(35000, 45)

### Get the Remaining Attacks

Set them all to have 35000 instances

In [215]:
df_DoS_attack = df_new[df_new['Attack']==7]
df_DoS_attack.shape

(500000, 45)

In [216]:
q1 = "DESCRIBE Encoded_L7_DstPrt_Cleaned_NF_dataset"
cursor.execute(q1)
colnames = [row[0] for row in cursor.fetchall()]

In [217]:
labels = [1,2,3,4,5,6,8,9,10,11,12,13,14]
data = []
for i in labels:
    cursor.execute("""SELECT * FROM Encoded_L7_DstPrt_Cleaned_NF_dataset WHERE Attack = %s ORDER BY RAND() LIMIT 35000""",(i,))
    rows = cursor.fetchall()
    data.extend(rows)

In [218]:
df = pd.DataFrame(data = data, columns = colnames)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_3306,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS
0,1168,5,4621,112,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
1,1672,5,1061,23,219,211,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0
2,1032,10,682,10,219,219,27,4294701,266,266,...,0,1,0,0,0,0,0,1,0,0
3,1076,10,682,10,219,219,27,4294716,251,251,...,0,1,0,0,0,0,0,1,0,0
4,529,5,341,5,219,219,27,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [219]:
np.unique(df['Attack'],return_counts=True)

(array([ 1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14]),
 array([  632,  1594,   819, 35000,  2108, 17171, 35000,    35,   266,
           47, 27031,   354, 15014]))

In [220]:
new_Benign.shape

(35000, 45)

In [221]:
df = pd.concat([new_Benign,df])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
153422,3131,24,7780,25,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
190153,2955,23,2086,24,27,27,27,4294952,15,15,...,0,0,0,0,0,0,0,0,1,0
219086,2102,19,8385,19,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
291063,1384,8,1801,9,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0
153639,65,1,123,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [222]:
df.shape

(170071, 45)

In [223]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14]),
 array([35000,   632,  1594,   819, 35000,  2108, 17171, 35000,    35,
          266,    47, 27031,   354, 15014]))

In [224]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]

In [225]:
smote = SMOTE(sampling_strategy = 'auto',random_state = 30)

In [226]:
X_resampled,y_resampled = smote.fit_resample(X,Y)

In [227]:
df = pd.concat([X_resampled,y_resampled],axis = 1)
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,3131,24,7780,25,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2955,23,2086,24,27,27,27,4294952,15,15,...,0,0,0,0,0,0,0,0,1,0
2,2102,19,8385,19,219,219,27,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1384,8,1801,9,30,30,26,0,0,0,...,0,0,1,0,0,0,0,0,1,0
4,65,1,123,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [228]:
np.unique(df['Attack'],return_counts = True)

(array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14]),
 array([35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000, 35000,
        35000, 35000, 35000, 35000, 35000]))

In [229]:
df_non = df
df_non['Attack'].values[:] = 0

In [230]:
df_non.shape

(490000, 45)

In [231]:
df = pd.concat([df_DoS_attack,df_non])
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,1388,12,2066,8,31,30,27,4294077,890,875,...,0,0,0,1,0,0,0,0,0,7
1,1574,10,1962,6,30,30,26,4294685,282,282,...,0,0,0,1,0,0,0,0,0,7
2,4924,19,2014,7,31,30,27,4294764,203,203,...,0,0,0,1,0,0,0,0,0,7
3,1345,10,2272,6,30,30,26,4294701,266,266,...,0,0,0,1,0,0,0,0,0,7
4,1100,10,1962,6,30,30,26,4294248,719,719,...,0,0,0,1,0,0,0,0,0,7


In [232]:
df.shape

(990000, 45)

In [233]:
np.unique(df['Attack'],return_counts = True)

(array([0, 7]), array([490000, 500000]))

### XGBoost Model of DoS_attacks_GoldenEye vs. Others

In [234]:
df.replace({'Attack':7},{'Attack':1},inplace = True)

In [235]:
df.head()

,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,DURATION_IN,DURATION_OUT,...,L4_DST_PORT_8080,L4_DST_PORT_31808,L4_DST_PORT_OTHERS,L7_PROTO_7,L7_PROTO_79,L7_PROTO_92,L7_PROTO_131,L7_PROTO_178,L7_PROTO_OTHERS,Attack
0,1388,12,2066,8,31,30,27,4294077,890,875,...,0,0,0,1,0,0,0,0,0,1
1,1574,10,1962,6,30,30,26,4294685,282,282,...,0,0,0,1,0,0,0,0,0,1
2,4924,19,2014,7,31,30,27,4294764,203,203,...,0,0,0,1,0,0,0,0,0,1
3,1345,10,2272,6,30,30,26,4294701,266,266,...,0,0,0,1,0,0,0,0,0,1
4,1100,10,1962,6,30,30,26,4294248,719,719,...,0,0,0,1,0,0,0,0,0,1


In [236]:
X = df.drop(columns = ['Attack'])
Y = df[['Attack']]
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state = 30)

In [268]:
model_DoS_attack = xgb.XGBClassifier()
# Train the model
model_DoS_attack.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [269]:
y_pred = model_DoS_attack.predict(X_test)

In [270]:
results = classification_report(y_test,y_pred,output_dict = True)

In [271]:
df = pd.DataFrame(results).transpose()

In [272]:
df

,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


### Random Forest Model of DoS_attacks_GoldenEye vs. Others

In [237]:
model_DoS_attack = RandomForestClassifier(n_estimators = 30, random_state = 30)
# Train the model
model_DoS_attack.fit(X_train,y_train)
y_pred = model_DoS_attack.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

/tmp/ipykernel_816085/1251113920.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_DoS_attack.fit(X_train,y_train)


,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0


In [238]:
model_pkl_file = "DoS Atacks-GoldenEye RF Model.pkl"  
with open(model_pkl_file, 'wb') as file:  
    pickle.dump(model_DoS_attack, file)

### DoS Attacks - GoldenEye LightGBM Model

In [274]:
model_DoS_attack = lgb.LGBMClassifier(objective="binary", random_state=42)
# Train the model
model_DoS_attack.fit(X_train, np.ravel(y_train))  # np.ravel() is used to convert the column vector y_train to a flattened array
y_pred = model_DoS_attack.predict(X_test)
results = classification_report(y_test,y_pred,output_dict = True)
df = pd.DataFrame(results).transpose()
df

[LightGBM] [Info] Number of positive: 399927, number of negative: 392073
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.013600 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4998
[LightGBM] [Info] Number of data points in the train set: 792000, number of used features: 42
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.504958 -> initscore=0.019834
[LightGBM] [Info] Start training from score 0.019834


,precision,recall,f1-score,support
0,1.0,1.0,1.0,97927.0
1,1.0,1.0,1.0,100073.0
accuracy,1.0,1.0,1.0,1.0
macro avg,1.0,1.0,1.0,198000.0
weighted avg,1.0,1.0,1.0,198000.0
